In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#pip install ydata-profiling==4.8.3

In [3]:
# pip install folium

In [4]:
#!{sys.executable} -m pip install -U ydata-profiling[notebook]

In [5]:
#pip install scipy==1.14.0

In [6]:
#pip install --force-reinstall numba==0.58.1

In [7]:

#pip install --ignore-installed numba==0.58.1

In [8]:
#import ProfileReport
#print(ProfileReport.__version__)

In [9]:
#imports of libs

from sklearn.linear_model import LogisticRegression, RidgeClassifierCV, Lasso


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#sklearn libs
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,classification_report,mean_squared_log_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

from scipy import stats as st

In [10]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
from sklearn.ensemble import RandomForestRegressor

In [12]:
#from ydata_profiling import ProfileReport

In [13]:
#profilling = ProfileReport(train)
#profilling

# **This notebook seeks to improve the regression analysis for the target column amount premium.**

First let's see the datasets 
* Load the datasets
* Look high cardinality columns
* Look the sample dataset description
* Look null coumn
* Look skewed colunms
* Look data type of column
* Look the outliers
* Look the Categorical columns and analize
* Look the bussiness holes
* Look the associations between columns and the target


In [14]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

sub = pd.read_csv("sample_submission.csv")

original_dataset_insurance = pd.read_csv('Insurance Premium Prediction Dataset.csv')

In [15]:
test.shape

(800000, 20)

In [16]:
train.describe()

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.200000e+06,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,5.999995e+05,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,3.464103e+05,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,0.000000e+00,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
25%,2.999998e+05,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,5.999995e+05,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,8.999992e+05,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
max,1.199999e+06,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


In [17]:
original_dataset_insurance.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,274175.000000,264905.000000,250974.000000,268263.000000,197572.000000,278860.000000,250974.000000,278860.000000,277019.000000
mean,41.020771,42089.085329,1.998048,28.584290,0.998117,9.520283,574.362049,5.007764,966.118667
std,13.549683,35444.517255,1.412312,15.966208,1.000795,5.767915,158.792037,2.581349,909.404567
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000
25%,29.000000,13588.000000,1.000000,16.149890,0.000000,5.000000,437.000000,3.000000,286.000000
50%,41.000000,32191.000000,2.000000,26.451244,1.000000,10.000000,575.000000,5.000000,688.000000
75%,53.000000,62164.000000,3.000000,38.966369,2.000000,15.000000,712.000000,7.000000,1367.000000
max,64.000000,149997.000000,4.000000,93.876090,9.000000,19.000000,849.000000,9.000000,4999.000000


In [18]:
original_dataset_insurance.dtypes

Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age               int64
Credit Score            float64
Insurance Duration        int64
Premium Amount          float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
dtype: object

In [19]:
train.dtypes

id                        int64
Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age             float64
Credit Score            float64
Insurance Duration      float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
Premium Amount          float64
dtype: object

Olhando os 3 códigos abaixo. Podemos verificar que os nomes de colunas dos dois datasets são iguais. A única diferença é a coluna id no dataset train.

In [20]:
train.columns

Index(['id', 'Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type', 'Premium Amount'],
      dtype='object')

In [21]:
original_dataset_insurance.columns

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Premium Amount',
       'Policy Start Date', 'Customer Feedback', 'Smoking Status',
       'Exercise Frequency', 'Property Type'],
      dtype='object')

In [22]:
train[original_dataset_insurance.columns]

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2869.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,1483.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,567.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,765.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2022.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,Premium,NaN,5.0,372.0,3.0,1303.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment
1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,Comprehensive,NaN,10.0,597.0,4.0,821.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment
1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,Basic,0.0,19.0,NaN,6.0,371.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo
1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,Premium,1.0,7.0,407.0,4.0,596.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment


Foi decidido filtrar, analisar e tratar os campos nulos. Como será tratado cada coluna / campo depende de análise uma a uma para que seja tomada a melhor decisão. Sempre levantar as questões: Porque aquele campo daquela coluna está vazio. Qual o significado daquele campo. Talvez o valor vazio pode representar um valor negativo, pode representar um valor novo não explorado no dataset, pode haver uma associação com outros campos. 

In [23]:
train.isnull().sum()

id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

In [24]:
original_dataset_insurance.isnull().sum()

Age                      4685
Gender                      0
Annual Income           13955
Marital Status           5019
Number of Dependents    27886
Education Level             0
Occupation              81288
Health Score            10597
Location                    0
Policy Type                 0
Previous Claims         81288
Vehicle Age                 0
Credit Score            27886
Insurance Duration          0
Premium Amount           1841
Policy Start Date           0
Customer Feedback       18349
Smoking Status              0
Exercise Frequency          0
Property Type               0
dtype: int64

*nota: Para facilitar o entendimento. Busquei o significado de cada coluna do dataset original.


Age: Age of the insured individual (Numerical)

In [25]:
comparison = pd.DataFrame(data=original_dataset_insurance['Age'])

In [26]:
comparison.describe

<bound method NDFrame.describe of          Age
0       56.0
1       46.0
2       32.0
3       60.0
4       25.0
...      ...
278855  53.0
278856  19.0
278857  45.0
278858  60.0
278859  50.0

[278860 rows x 1 columns]>

In [27]:
comparison['train_age'] = train['Age'] 

In [28]:
comparison['train_age']

0         19.0
1         39.0
2         23.0
3         21.0
4         21.0
          ... 
278855    21.0
278856    38.0
278857    57.0
278858    19.0
278859    45.0
Name: train_age, Length: 278860, dtype: float64

In [29]:
comparison['compare_age'] = comparison['train_age'] == comparison['Age']

In [30]:
#Tentei confrontar o mesmo campo das duas bases e ficou claro que são diferentes. 
#Não consigo preencher uma base com a coluna de outra.
comparison['compare_age'] == True

0         False
1         False
2         False
3         False
4         False
          ...  
278855    False
278856    False
278857    False
278858    False
278859    False
Name: compare_age, Length: 278860, dtype: bool

In [31]:
comparison

,Age,train_age,compare_age
0,56.0,19.0,False
1,46.0,39.0,False
2,32.0,23.0,False
3,60.0,21.0,False
4,25.0,21.0,False
...,...,...,...
278855,53.0,21.0,False
278856,19.0,38.0,False
278857,45.0,57.0,False
278858,60.0,19.0,False


In [32]:
np.sort( train['Age'].unique() )

array([18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.,
       31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43.,
       44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56.,
       57., 58., 59., 60., 61., 62., 63., 64., nan])

Olhando em ordem ascendente, preciso descobrir associações entre atributos que possam indicar a idade ou, caso seja necessário, a faixa de idade do cliente.

In [33]:
train[~train['Age'].isnull()]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,NaN,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199996,1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,...,NaN,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,...,0.0,19.0,NaN,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


Olhando para o Health Score por faixas de idade consegui identificar que apenas na faixa entre 40 e 50 anos que há o valor Nan que significa vazio. Será feito a mediana dos valores dets faixa de idade para preencher os valores nulo.

In [34]:
health_score_age_40_50 = train[ (~train['Age'].isnull() ) & (train['Age'] > 40) & (train['Age'] < 50) ]['Health Score'].median()
health_score_age_40_50

24.50280053089128

In [35]:
health_score_age_40_50_test = test[ (~test['Age'].isnull() ) & (test['Age'] > 40) & (test['Age'] < 50) ]['Health Score'].median()
health_score_age_40_50_test

24.47178150322004

Health Score: A score representing the health status (Numerical, skewed)

In [36]:
train['Health Score'].isna()

0          False
1          False
2          False
3          False
4          False
           ...  
1199995    False
1199996    False
1199997    False
1199998    False
1199999    False
Name: Health Score, Length: 1200000, dtype: bool

In [37]:
train[ train['Health Score'].isna() == True ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
6,6,41.0,Male,40336.0,Married,0.0,PhD,NaN,NaN,Rural,...,2.0,8.0,807.0,6.0,2020-02-21 15:21:39.219432,Poor,No,Weekly,House,439.0
10,10,56.0,Female,8054.0,Married,1.0,Bachelor's,Unemployed,NaN,Rural,...,1.0,8.0,431.0,8.0,2021-05-12 15:21:39.134960,Average,No,Monthly,Condo,857.0
51,51,59.0,Male,523.0,Divorced,1.0,PhD,NaN,NaN,Urban,...,1.0,9.0,439.0,9.0,2021-01-02 15:21:39.248098,Average,Yes,Weekly,Apartment,537.0
69,69,22.0,Female,73693.0,Married,2.0,Master's,Self-Employed,NaN,Rural,...,1.0,1.0,498.0,1.0,2023-07-01 15:21:39.134960,Poor,Yes,Weekly,Apartment,588.0
70,70,62.0,Male,11977.0,Married,2.0,PhD,Employed,NaN,Suburban,...,3.0,0.0,NaN,8.0,2021-12-31 15:21:39.167099,Poor,Yes,Monthly,House,690.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199940,1199940,57.0,Female,6871.0,Married,NaN,Bachelor's,NaN,NaN,Rural,...,2.0,19.0,456.0,5.0,2022-01-25 15:21:39.151732,Good,Yes,Rarely,Apartment,1675.0
1199964,1199964,29.0,Female,72882.0,Single,1.0,High School,Unemployed,NaN,Rural,...,NaN,6.0,507.0,7.0,2022-03-19 15:21:39.281787,Good,Yes,Daily,House,1063.0
1199967,1199967,18.0,Female,111949.0,Divorced,2.0,High School,Employed,NaN,Urban,...,0.0,6.0,460.0,2.0,2023-07-29 15:21:39.134960,Average,No,Daily,Apartment,922.0
1199979,1199979,42.0,Male,44691.0,Married,NaN,Bachelor's,NaN,NaN,Suburban,...,NaN,7.0,516.0,8.0,2019-10-10 15:21:39.134960,Poor,Yes,Rarely,Apartment,1358.0


In [38]:
train['Health Score'][ train['Health Score'].isnull() == True ] = health_score_age_40_50

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\3241939718.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train['Health Score'][ train['Health Score'].isnull() == True ] = health_score_age_40_50
C:\Users\makrr\AppData\Local\Temp\ipyke

In [39]:
test['Health Score'][ test['Health Score'].isnull() == True ] = health_score_age_40_50_test

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\3190559845.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test['Health Score'][ test['Health Score'].isnull() == True ] = health_score_age_40_50_test
C:\Users\makrr\AppData\Local\Temp\ip

Logo abaixo está sendo verificado se a alteração está nos conformes. Deve trazer a linha que carrega o id 1199989 que estava vazia o valor definido acima.

In [40]:
train[train['id'] == 1199989 ]['Health Score']

1199989    24.502801
Name: Health Score, dtype: float64

In [41]:
train[train['Health Score'].isnull() == True]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount


In [42]:
train[ (~train['Age'].isnull() ) & (train['Age'] > 40) & (train['Age'] < 50) ]['Health Score'].unique()

array([24.50280053,  5.76978309, 20.47371786, ..., 15.24242508,
       15.75150064, 19.14509743])

In [43]:
#train[ (~train['Age'].isnull() ) & (train['Age'] > 40) & (train['Age'] < 50) & (train['Health Score'] == 'nan' ) ]['Health Score']

In [44]:
#.astype('int64').median().astype(int)

In [45]:
age_string = train[~train['Age'].isnull()]['Age'].astype('str')

In [46]:
train['Age'].isnull()

0          False
1          False
2          False
3          False
4          False
           ...  
1199995    False
1199996    False
1199997    False
1199998    False
1199999    False
Name: Age, Length: 1200000, dtype: bool

In [47]:
train[~ train['Age'].isnull() == True ]['Age'].dtypes

dtype('float64')

In [48]:
#pd.to_numeric(train[~ train['Age'].isnull() == True ]['Age'] , errors='coerce')

In [49]:
health_score_age_40_50

24.50280053089128

In [50]:
train['Age'].fillna(value=health_score_age_40_50, inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\2659824241.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Age'].fillna(value=health_score_age_40_50, inplace=True)


In [51]:
test['Age'].fillna(value=health_score_age_40_50_test, inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\2919329032.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Age'].fillna(value=health_score_age_40_50_test, inplace=True)


In [52]:
train[ (~train['Age'].isnull() ) & (train['Age'] > 19) & (train['Age'] < 30) ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
5,5,29.0,Male,45963.0,Married,1.0,Bachelor's,NaN,33.053198,Urban,...,2.0,4.0,614.0,5.0,2022-05-20 15:21:39.207847,Average,No,Weekly,House,3202.0
8,8,21.0,Male,1733.0,Divorced,3.0,Bachelor's,NaN,17.869551,Urban,...,1.0,10.0,685.0,8.0,2020-12-14 15:21:39.198406,Average,No,Monthly,Condo,213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199976,1199976,24.0,Female,937.0,Divorced,4.0,High School,NaN,43.843190,Urban,...,2.0,9.0,748.0,9.0,2021-09-22 15:21:39.217387,Poor,No,Weekly,Apartment,1568.0
1199981,1199981,20.0,Female,23685.0,Single,1.0,High School,Self-Employed,15.173435,Suburban,...,NaN,14.0,441.0,8.0,2023-05-07 15:21:39.237118,Average,No,Weekly,House,101.0
1199984,1199984,20.0,Female,16582.0,Divorced,2.0,Master's,Employed,39.756904,Rural,...,NaN,8.0,633.0,3.0,2020-03-21 15:21:39.134960,Poor,No,Daily,Apartment,524.0
1199987,1199987,25.0,Female,4089.0,Single,1.0,Bachelor's,Unemployed,12.335586,Urban,...,0.0,17.0,557.0,1.0,2022-12-02 15:21:39.079128,NaN,Yes,Monthly,House,473.0


In [53]:
train[ train['Age'].isnull() == True ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount


In [54]:
train[ (~train['Age'].isnull() ) & (train['Age'] > 19) & (train['Age'] < 30) ]['Health Score'].unique()

array([47.17754929, 10.93814416, 20.37609363, ..., 43.8431898 ,
       39.75690435, 12.33558635])

In [55]:
train[ (~train['Age'].isnull() ) & (train['Age'] > 30) & (train['Age'] < 40) ]['Health Score'].unique()

array([15.56973099, 24.05963876, 17.81445892, ..., 22.03692707,
       30.38994596, 14.78343851])

In [56]:
train[ (~train['Age'].isnull() ) & (train['Age'] > 40) & (train['Age'] < 50) ]['Health Score'].unique()

array([24.50280053,  5.76978309, 20.47371786, ..., 15.24242508,
       15.75150064, 19.14509743])

In [57]:
train[ (~train['Age'].isnull() ) & (train['Age'] > 60) ]['Health Score'].unique()

array([20.71707647, 40.88612373, 24.12174595, ..., 23.52718114,
       23.6778361 , 14.87328325])

In [58]:
#original_dataset_insurance.astype('int32').dtypes

In [59]:
original_dataset_insurance.shape

(278860, 20)

In [60]:
train.shape

(1200000, 21)

In [61]:
train.isnull().sum()

id                           0
Age                          0
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score                 0
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

In [62]:
original_dataset_insurance.isnull().sum()

Age                      4685
Gender                      0
Annual Income           13955
Marital Status           5019
Number of Dependents    27886
Education Level             0
Occupation              81288
Health Score            10597
Location                    0
Policy Type                 0
Previous Claims         81288
Vehicle Age                 0
Credit Score            27886
Insurance Duration          0
Premium Amount           1841
Policy Start Date           0
Customer Feedback       18349
Smoking Status              0
Exercise Frequency          0
Property Type               0
dtype: int64

In [63]:
train['Customer Feedback'].unique()

array(['Poor', 'Average', 'Good', nan], dtype=object)

In [64]:
train[train['Customer Feedback'].isna() == True]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
28,28,43.0,Male,1060.0,Married,NaN,Master's,Self-Employed,41.376716,Suburban,...,0.0,8.0,795.0,7.0,2019-09-06 15:21:39.171102,NaN,Yes,Rarely,Condo,699.0
59,59,62.0,Female,147020.0,Married,4.0,Bachelor's,Self-Employed,24.121746,Urban,...,NaN,0.0,498.0,6.0,2021-05-10 15:21:39.155231,NaN,No,Daily,Condo,1624.0
60,60,56.0,Female,122824.0,Divorced,3.0,Master's,NaN,41.938170,Urban,...,1.0,6.0,662.0,1.0,2020-02-05 15:21:39.133866,NaN,Yes,Weekly,Apartment,2932.0
79,79,27.0,Male,948.0,Single,3.0,Master's,NaN,37.479541,Rural,...,NaN,16.0,534.0,9.0,2020-06-24 15:21:39.216387,NaN,No,Rarely,House,514.0
90,90,19.0,Female,68892.0,Married,3.0,Bachelor's,Employed,41.802670,Urban,...,NaN,3.0,795.0,1.0,2022-10-15 15:21:39.177724,NaN,No,Monthly,House,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199911,1199911,61.0,Male,11836.0,Married,4.0,Bachelor's,Self-Employed,29.395499,Rural,...,2.0,6.0,NaN,9.0,2021-05-26 15:21:39.237118,NaN,Yes,Monthly,Condo,240.0
1199917,1199917,63.0,Male,44159.0,Single,1.0,High School,Unemployed,23.527181,Rural,...,1.0,14.0,NaN,5.0,2024-02-25 15:21:39.079128,NaN,Yes,Daily,Apartment,3522.0
1199949,1199949,19.0,Female,968.0,Married,2.0,Bachelor's,Employed,22.738222,Rural,...,0.0,12.0,662.0,4.0,2021-10-18 15:21:39.155231,NaN,No,Weekly,Apartment,1721.0
1199966,1199966,38.0,Female,41035.0,Divorced,2.0,Bachelor's,NaN,30.389946,Suburban,...,0.0,0.0,579.0,9.0,2020-06-06 15:21:39.254678,NaN,Yes,Daily,Condo,1672.0


In [65]:
train['Customer Feedback'].fillna(value='Average', inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\1395459927.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Customer Feedback'].fillna(value='Average', inplace=True)


In [66]:
np.sort( train['Credit Score'].unique() )

array([300., 301., 302., 303., 304., 305., 306., 307., 308., 309., 310.,
       311., 312., 313., 314., 315., 316., 317., 318., 319., 320., 321.,
       322., 323., 324., 325., 326., 327., 328., 329., 330., 331., 332.,
       333., 334., 335., 336., 337., 338., 339., 340., 341., 342., 343.,
       344., 345., 346., 347., 348., 349., 350., 351., 352., 353., 354.,
       355., 356., 357., 358., 359., 360., 361., 362., 363., 364., 365.,
       366., 367., 368., 369., 370., 371., 372., 373., 374., 375., 376.,
       377., 378., 379., 380., 381., 382., 383., 384., 385., 386., 387.,
       388., 389., 390., 391., 392., 393., 394., 395., 396., 397., 398.,
       399., 400., 401., 402., 403., 404., 405., 406., 407., 408., 409.,
       410., 411., 412., 413., 414., 415., 416., 417., 418., 419., 420.,
       421., 422., 423., 424., 425., 426., 427., 428., 429., 430., 431.,
       432., 433., 434., 435., 436., 437., 438., 439., 440., 441., 442.,
       443., 444., 445., 446., 447., 448., 449., 45

In [67]:
credit_score_median = train['Credit Score'].unique()

In [68]:
credit_score_median_test = test['Credit Score'].unique()

In [69]:
credit_score_median = credit_score_median[~np.isnan(credit_score_median)]
credit_score_median

array([372., 694., 367., 598., 614., 807., 398., 685., 635., 431., 597.,
       511., 498., 584., 420., 595., 799., 773., 798., 543., 416., 425.,
       486., 795., 495., 449., 803., 421., 695., 713., 589., 812., 382.,
       469., 530., 538., 842., 575., 794., 836., 784., 439., 381., 801.,
       624., 497., 359., 662., 703., 426., 331., 679., 375., 321., 531.,
       622., 464., 418., 534., 454., 502., 823., 319., 660., 371., 562.,
       488., 577., 791., 638., 438., 779., 310., 748., 392., 446., 593.,
       576., 493., 479., 826., 332., 616., 716., 602., 343., 377., 788.,
       758., 571., 554., 682., 641., 360., 522., 728., 388., 535., 519.,
       651., 829., 620., 776., 835., 432., 607., 705., 644., 686., 797.,
       509., 831., 529., 774., 592., 843., 738., 781., 766., 364., 349.,
       692., 574., 324., 632., 379., 434., 442., 764., 429., 683., 770.,
       696., 611., 436., 482., 636., 609., 374., 566., 537., 648., 578.,
       707., 581., 541., 845., 441., 718., 630., 58

In [70]:
credit_score_median_test = credit_score_median_test[~np.isnan(credit_score_median_test)]
credit_score_median_test

array([372., 819., 770., 755., 432., 321., 712., 540., 757., 408., 849.,
       818., 621., 319., 542., 602., 595., 468., 318., 382., 515., 731.,
       388., 466., 758., 552., 797., 699., 324., 524., 727., 640., 719.,
       496., 766., 503., 513., 498., 411., 619., 704., 614., 809., 697.,
       653., 584., 626., 826., 813., 662., 537., 438., 544., 782., 683.,
       590., 677., 489., 617., 759., 734., 612., 781., 455., 787., 711.,
       585., 593., 749., 543., 607., 361., 821., 454., 672., 500., 373.,
       338., 558., 490., 796., 656., 802., 622., 591., 690., 817., 641.,
       615., 548., 816., 627., 538., 726., 566., 366., 788., 636., 642.,
       479., 476., 778., 441., 721., 314., 682., 718., 687., 421., 795.,
       790., 723., 774., 482., 449., 553., 406., 354., 812., 696., 639.,
       743., 356., 838., 518., 486., 462., 685., 744., 404., 522., 733.,
       395., 710., 471., 477., 511., 370., 767., 807., 325., 632., 453.,
       709., 434., 451., 786., 803., 428., 799., 77

In [71]:
credit_score_median = np.median( credit_score_median )

In [72]:
credit_score_mode = st.mode( credit_score_median )
credit_score_mode

ModeResult(mode=574.5, count=1)

In [73]:
credit_score_mode_test = st.mode( credit_score_median_test )
credit_score_mode_test

ModeResult(mode=300.0, count=1)

In [74]:
train['Credit Score'].unique()

array([372., 694.,  nan, 367., 598., 614., 807., 398., 685., 635., 431.,
       597., 511., 498., 584., 420., 595., 799., 773., 798., 543., 416.,
       425., 486., 795., 495., 449., 803., 421., 695., 713., 589., 812.,
       382., 469., 530., 538., 842., 575., 794., 836., 784., 439., 381.,
       801., 624., 497., 359., 662., 703., 426., 331., 679., 375., 321.,
       531., 622., 464., 418., 534., 454., 502., 823., 319., 660., 371.,
       562., 488., 577., 791., 638., 438., 779., 310., 748., 392., 446.,
       593., 576., 493., 479., 826., 332., 616., 716., 602., 343., 377.,
       788., 758., 571., 554., 682., 641., 360., 522., 728., 388., 535.,
       519., 651., 829., 620., 776., 835., 432., 607., 705., 644., 686.,
       797., 509., 831., 529., 774., 592., 843., 738., 781., 766., 364.,
       349., 692., 574., 324., 632., 379., 434., 442., 764., 429., 683.,
       770., 696., 611., 436., 482., 636., 609., 374., 566., 537., 648.,
       578., 707., 581., 541., 845., 441., 718., 63

In [75]:
train['Credit Score'].median()

595.0

In [76]:
train['Credit Score'].mode()

0    434.0
Name: Credit Score, dtype: float64

In [77]:
train[train['Credit Score'].isna() == True]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
12,12,25.0,Female,23706.0,Single,4.0,Master's,Employed,4.090538,Urban,...,2.0,19.0,NaN,2.0,2021-10-28 15:21:39.086123,Poor,Yes,Weekly,Apartment,703.0
16,16,59.0,Female,28266.0,Divorced,2.0,PhD,Unemployed,21.673461,Urban,...,0.0,16.0,NaN,3.0,2024-04-22 15:21:39.255696,Average,Yes,Weekly,Condo,183.0
18,18,18.0,Male,29071.0,Married,NaN,Bachelor's,Employed,20.838977,Rural,...,1.0,5.0,NaN,3.0,2021-10-13 15:21:39.094752,Poor,Yes,Rarely,Apartment,787.0
31,31,37.0,Male,4534.0,Divorced,NaN,Bachelor's,NaN,26.387481,Suburban,...,0.0,1.0,NaN,1.0,2022-09-21 15:21:39.102694,Poor,No,Daily,House,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199957,1199957,32.0,Female,16221.0,Divorced,1.0,High School,NaN,28.695917,Urban,...,1.0,15.0,NaN,6.0,2023-11-27 15:21:39.206847,Good,No,Monthly,Apartment,537.0
1199959,1199959,48.0,Male,1167.0,Married,3.0,PhD,Employed,15.751501,Suburban,...,0.0,11.0,NaN,4.0,2024-06-16 15:21:39.278402,Good,Yes,Monthly,Condo,28.0
1199989,1199989,18.0,Female,29092.0,Single,2.0,High School,Employed,24.502801,Rural,...,NaN,14.0,NaN,1.0,2023-03-17 15:21:39.106582,Poor,Yes,Rarely,Apartment,543.0
1199991,1199991,59.0,Female,23706.0,Divorced,4.0,High School,Self-Employed,24.913204,Suburban,...,NaN,17.0,NaN,1.0,2021-06-22 15:21:39.188220,Good,Yes,Monthly,Apartment,3381.0


In [78]:
train[train['Credit Score'].isna() == True]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
12,12,25.0,Female,23706.0,Single,4.0,Master's,Employed,4.090538,Urban,...,2.0,19.0,NaN,2.0,2021-10-28 15:21:39.086123,Poor,Yes,Weekly,Apartment,703.0
16,16,59.0,Female,28266.0,Divorced,2.0,PhD,Unemployed,21.673461,Urban,...,0.0,16.0,NaN,3.0,2024-04-22 15:21:39.255696,Average,Yes,Weekly,Condo,183.0
18,18,18.0,Male,29071.0,Married,NaN,Bachelor's,Employed,20.838977,Rural,...,1.0,5.0,NaN,3.0,2021-10-13 15:21:39.094752,Poor,Yes,Rarely,Apartment,787.0
31,31,37.0,Male,4534.0,Divorced,NaN,Bachelor's,NaN,26.387481,Suburban,...,0.0,1.0,NaN,1.0,2022-09-21 15:21:39.102694,Poor,No,Daily,House,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199957,1199957,32.0,Female,16221.0,Divorced,1.0,High School,NaN,28.695917,Urban,...,1.0,15.0,NaN,6.0,2023-11-27 15:21:39.206847,Good,No,Monthly,Apartment,537.0
1199959,1199959,48.0,Male,1167.0,Married,3.0,PhD,Employed,15.751501,Suburban,...,0.0,11.0,NaN,4.0,2024-06-16 15:21:39.278402,Good,Yes,Monthly,Condo,28.0
1199989,1199989,18.0,Female,29092.0,Single,2.0,High School,Employed,24.502801,Rural,...,NaN,14.0,NaN,1.0,2023-03-17 15:21:39.106582,Poor,Yes,Rarely,Apartment,543.0
1199991,1199991,59.0,Female,23706.0,Divorced,4.0,High School,Self-Employed,24.913204,Suburban,...,NaN,17.0,NaN,1.0,2021-06-22 15:21:39.188220,Good,Yes,Monthly,Apartment,3381.0


In [79]:
train['Credit Score']

0          372.0
1          694.0
2            NaN
3          367.0
4          598.0
           ...  
1199995    372.0
1199996    597.0
1199997      NaN
1199998    407.0
1199999    502.0
Name: Credit Score, Length: 1200000, dtype: float64

In [80]:
train['Credit Score'].fillna(value=credit_score_mode.mode, inplace=True)

In [81]:
test['Credit Score'].fillna(value=credit_score_mode_test.mode, inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\3136939871.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Credit Score'].fillna(value=credit_score_mode_test.mode, inplace=True)


No Overview mostra que o modelo será medido pelo método Mean Square Log Error. Por isto que vou utilizá-lo no final.

In [82]:
train.isnull().sum()

id                           0
Age                          0
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score                 0
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score                 0
Insurance Duration           1
Policy Start Date            0
Customer Feedback            0
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

Previous Claims: Number of previous claims made (Numerical, with outliers)

In [83]:
train['Previous Claims']

0          2.0
1          1.0
2          1.0
3          1.0
4          0.0
          ... 
1199995    NaN
1199996    NaN
1199997    0.0
1199998    1.0
1199999    0.0
Name: Previous Claims, Length: 1200000, dtype: float64

In [84]:
train['Previous Claims'].unique()

array([ 2.,  1.,  0., nan,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

In [85]:
previous_claim  = train['Previous Claims'].mode()[0]
previous_claim

0.0

In [86]:
previous_claim_test  = test['Previous Claims'].mode()[0]
previous_claim_test

0.0

In [87]:
train['Previous Claims'].fillna(value=previous_claim, inplace=True)

In [88]:
test['Previous Claims'].fillna(value=previous_claim_test, inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\4166880948.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Previous Claims'].fillna(value=previous_claim_test, inplace=True)


In [89]:
train['Previous Claims'].unique()

array([2., 1., 0., 3., 4., 5., 6., 7., 8., 9.])

In [90]:
train[train['Previous Claims'] == 0]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
11,11,23.0,Male,30983.0,Single,3.0,Master's,NaN,5.813129,Urban,...,0.0,6.0,597.0,8.0,2020-03-22 15:21:39.155231,Good,No,Rarely,Condo,1447.0
13,13,44.0,Female,70893.0,Single,0.0,Bachelor's,NaN,55.896322,Suburban,...,0.0,3.0,511.0,6.0,2023-01-20 15:21:39.132191,Good,Yes,Weekly,House,1847.0
16,16,59.0,Female,28266.0,Divorced,2.0,PhD,Unemployed,21.673461,Urban,...,0.0,16.0,574.5,3.0,2024-04-22 15:21:39.255696,Average,Yes,Weekly,Condo,183.0
17,17,34.0,Female,45907.0,Divorced,4.0,High School,Self-Employed,24.059639,Suburban,...,0.0,0.0,694.0,8.0,2020-12-27 15:21:39.090124,Poor,Yes,Daily,Condo,643.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199994,1199994,34.0,Male,23456.0,Single,4.0,Master's,Self-Employed,14.783439,Rural,...,0.0,12.0,548.0,9.0,2023-06-09 15:21:39.134960,Good,No,Monthly,Apartment,1584.0
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,0.0,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199996,1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,...,0.0,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,...,0.0,19.0,574.5,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0


In [91]:
train[train['Previous Claims'].isna() == True]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount


In [92]:
train.isnull().sum()

id                           0
Age                          0
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score                 0
Location                     0
Policy Type                  0
Previous Claims              0
Vehicle Age                  6
Credit Score                 0
Insurance Duration           1
Policy Start Date            0
Customer Feedback            0
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

Occupation: Occupation of the insured individual (Categorical: Employed, Self-Employed, Unemployed)

In [93]:
train[ train['Occupation'].isnull() == True ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
5,5,29.0,Male,45963.0,Married,1.0,Bachelor's,NaN,33.053198,Urban,...,2.0,4.0,614.0,5.0,2022-05-20 15:21:39.207847,Average,No,Weekly,House,3202.0
6,6,41.0,Male,40336.0,Married,0.0,PhD,NaN,24.502801,Rural,...,2.0,8.0,807.0,6.0,2020-02-21 15:21:39.219432,Poor,No,Weekly,House,439.0
8,8,21.0,Male,1733.0,Divorced,3.0,Bachelor's,NaN,17.869551,Urban,...,1.0,10.0,685.0,8.0,2020-12-14 15:21:39.198406,Average,No,Monthly,Condo,213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199983,1199983,43.0,Female,4518.0,Divorced,0.0,High School,NaN,19.145097,Suburban,...,3.0,12.0,489.0,4.0,2022-08-14 15:21:39.254678,Poor,No,Weekly,Condo,573.0
1199993,1199993,38.0,Male,1607.0,Married,1.0,High School,NaN,18.552314,Suburban,...,0.0,12.0,469.0,2.0,2022-08-10 15:21:39.132191,Good,No,Rarely,House,1027.0
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,...,0.0,19.0,574.5,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


Analisando que existe uma coluna que trás a renda anual. Quando houve valores vazios neste campo pode - se inferir que esteja desempregado.

In [94]:
train[ (train['Annual Income'].isnull() == True) & (train['Occupation'].isnull() == True) ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
22,22,22.0,Male,NaN,Divorced,4.0,PhD,NaN,25.583790,Urban,...,0.0,5.0,773.0,5.0,2021-10-09 15:21:39.258696,Good,Yes,Monthly,House,202.0
86,86,37.0,Male,NaN,Single,1.0,Bachelor's,NaN,13.038269,Suburban,...,0.0,6.0,562.0,2.0,2023-02-02 15:21:39.237118,Average,No,Weekly,House,61.0
101,101,39.0,Female,NaN,Married,0.0,High School,NaN,24.534293,Suburban,...,0.0,7.0,574.5,5.0,2024-04-23 15:21:39.155231,Good,No,Daily,House,164.0
427,427,45.0,Female,NaN,Married,0.0,High School,NaN,40.835226,Rural,...,0.0,5.0,574.5,9.0,2021-06-27 15:21:39.199447,Average,Yes,Monthly,House,251.0
503,503,34.0,Male,NaN,Single,4.0,Bachelor's,NaN,18.797790,Suburban,...,0.0,11.0,758.0,2.0,2021-05-13 15:21:39.279729,Poor,Yes,Weekly,Condo,399.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199757,1199757,46.0,Female,NaN,Married,3.0,High School,NaN,17.869551,Urban,...,2.0,18.0,439.0,7.0,2022-01-15 15:21:39.086123,Average,Yes,Daily,Condo,1438.0
1199852,1199852,41.0,Male,NaN,Divorced,NaN,Bachelor's,NaN,13.088557,Suburban,...,1.0,15.0,675.0,1.0,2024-05-06 15:21:39.102694,Average,Yes,Weekly,Condo,212.0
1199875,1199875,41.0,Male,NaN,Divorced,2.0,Bachelor's,NaN,27.891604,Rural,...,1.0,14.0,587.0,7.0,2023-11-30 15:21:39.274993,Average,No,Daily,Apartment,2792.0
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


In [95]:
train['Occupation'].unique()

array(['Self-Employed', nan, 'Employed', 'Unemployed'], dtype=object)

In [96]:
train['Occupation'].mode()

0    Employed
Name: Occupation, dtype: object

In [97]:
#Pelo visto não há valor 0. 
#Talvez seja um indicativo que sempre quando este campo for vazio pode ser que seja sem renda.
train[train['Annual Income'] == 0]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount


Pela análise feita até agora neste campo. Acredito que Quando não há renda anual. O cliente está desempregado. Por isto será imputado o valor 'Unemployed' quando a renda for nan.

In [98]:
train.loc[
    (train['Annual Income'].isnull()) & (train['Occupation'].isnull()), 
    'Occupation'
] = 'Unemployed'

In [99]:
test.loc[
    (test['Annual Income'].isnull()) & (test['Occupation'].isnull()), 
    'Occupation'
] = 'Unemployed'

Vamos verificar se o campo Occupation está preenchido. O id 22 não estava preenchido. Vamos usar como exemplo.

In [100]:
train[ train['id'] == 22 ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
22,22,22.0,Male,NaN,Divorced,4.0,PhD,Unemployed,25.58379,Urban,...,0.0,5.0,773.0,5.0,2021-10-09 15:21:39.258696,Good,Yes,Monthly,House,202.0


In [101]:
train['Occupation'].fillna( value=train['Occupation'].mode()[0], inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\3142172295.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Occupation'].fillna( value=train['Occupation'].mode()[0], inplace=True)


In [102]:
test['Occupation'].fillna( value=test['Occupation'].mode()[0], inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\284626330.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Occupation'].fillna( value=test['Occupation'].mode()[0], inplace=True)


In [103]:
train[ (train['Annual Income'].isnull() == True) ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
22,22,22.0,Male,NaN,Divorced,4.0,PhD,Unemployed,25.583790,Urban,...,0.0,5.0,773.0,5.0,2021-10-09 15:21:39.258696,Good,Yes,Monthly,House,202.0
36,36,41.0,Female,NaN,Married,3.0,PhD,Self-Employed,14.001630,Urban,...,0.0,15.0,589.0,6.0,2023-03-01 15:21:39.149731,Average,No,Rarely,House,250.0
67,67,45.0,Male,NaN,Married,3.0,High School,Self-Employed,17.988631,Suburban,...,0.0,18.0,375.0,9.0,2022-05-25 15:21:39.237118,Average,Yes,Rarely,House,829.0
86,86,37.0,Male,NaN,Single,1.0,Bachelor's,Unemployed,13.038269,Suburban,...,0.0,6.0,562.0,2.0,2023-02-02 15:21:39.237118,Average,No,Weekly,House,61.0
87,87,52.0,Male,NaN,Married,2.0,PhD,Unemployed,28.864410,Rural,...,0.0,18.0,488.0,1.0,2021-02-13 15:21:39.172097,Average,No,Weekly,Apartment,369.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199875,1199875,41.0,Male,NaN,Divorced,2.0,Bachelor's,Unemployed,27.891604,Rural,...,1.0,14.0,587.0,7.0,2023-11-30 15:21:39.274993,Average,No,Daily,Apartment,2792.0
1199894,1199894,30.0,Female,NaN,Single,3.0,Bachelor's,Unemployed,29.844105,Urban,...,1.0,7.0,400.0,8.0,2021-04-25 15:21:39.257696,Average,Yes,Rarely,House,1087.0
1199925,1199925,45.0,Male,NaN,Married,0.0,High School,Self-Employed,51.402797,Urban,...,0.0,2.0,574.5,8.0,2023-06-30 15:21:39.263607,Good,No,Rarely,Condo,1526.0
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


Agora será feito a associação entre as colunas 'Annual Income' e Occupation. O registro que a coluna estiver 'Unemployed' será inputado 0.

In [104]:
train[ (train['Annual Income'].isnull() == True) & (train['Occupation'] == 'Unemployed') ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
22,22,22.0,Male,NaN,Divorced,4.0,PhD,Unemployed,25.583790,Urban,...,0.0,5.0,773.0,5.0,2021-10-09 15:21:39.258696,Good,Yes,Monthly,House,202.0
86,86,37.0,Male,NaN,Single,1.0,Bachelor's,Unemployed,13.038269,Suburban,...,0.0,6.0,562.0,2.0,2023-02-02 15:21:39.237118,Average,No,Weekly,House,61.0
87,87,52.0,Male,NaN,Married,2.0,PhD,Unemployed,28.864410,Rural,...,0.0,18.0,488.0,1.0,2021-02-13 15:21:39.172097,Average,No,Weekly,Apartment,369.0
101,101,39.0,Female,NaN,Married,0.0,High School,Unemployed,24.534293,Suburban,...,0.0,7.0,574.5,5.0,2024-04-23 15:21:39.155231,Good,No,Daily,House,164.0
159,159,60.0,Female,NaN,Single,4.0,Master's,Unemployed,36.331482,Suburban,...,0.0,8.0,574.5,8.0,2023-08-30 15:21:39.283783,Poor,Yes,Rarely,Apartment,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199852,1199852,41.0,Male,NaN,Divorced,NaN,Bachelor's,Unemployed,13.088557,Suburban,...,1.0,15.0,675.0,1.0,2024-05-06 15:21:39.102694,Average,Yes,Weekly,Condo,212.0
1199875,1199875,41.0,Male,NaN,Divorced,2.0,Bachelor's,Unemployed,27.891604,Rural,...,1.0,14.0,587.0,7.0,2023-11-30 15:21:39.274993,Average,No,Daily,Apartment,2792.0
1199894,1199894,30.0,Female,NaN,Single,3.0,Bachelor's,Unemployed,29.844105,Urban,...,1.0,7.0,400.0,8.0,2021-04-25 15:21:39.257696,Average,Yes,Rarely,House,1087.0
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


In [105]:
train.loc[ (train['Annual Income'].isnull() == True) & (train['Occupation'] == 'Unemployed') ,'Annual Income'] = 0

In [106]:
test.loc[ (test['Annual Income'].isnull() == True) & (test['Occupation'] == 'Unemployed') ,'Annual Income'] = 0

In [107]:
train[train['id'] == 22 ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
22,22,22.0,Male,0.0,Divorced,4.0,PhD,Unemployed,25.58379,Urban,...,0.0,5.0,773.0,5.0,2021-10-09 15:21:39.258696,Good,Yes,Monthly,House,202.0


In [108]:
train.isnull().sum()

id                           0
Age                          0
Gender                       0
Annual Income            21252
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation                   0
Health Score                 0
Location                     0
Policy Type                  0
Previous Claims              0
Vehicle Age                  6
Credit Score                 0
Insurance Duration           1
Policy Start Date            0
Customer Feedback            0
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

Number of Dependents: Number of dependents (Numerical, with missing values)

In [109]:
train[~train['Number of Dependents'].isnull()]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,574.5,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199994,1199994,34.0,Male,23456.0,Single,4.0,Master's,Self-Employed,14.783439,Rural,...,0.0,12.0,548.0,9.0,2023-06-09 15:21:39.134960,Good,No,Monthly,Apartment,1584.0
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,0.0,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,0.0,19.0,574.5,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


In [110]:
train[train['Number of Dependents'].isnull()]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
14,14,40.0,Female,23897.0,Divorced,NaN,High School,Self-Employed,29.082036,Suburban,...,2.0,15.0,498.0,1.0,2021-08-14 15:21:39.233998,Good,No,Rarely,Condo,30.0
18,18,18.0,Male,29071.0,Married,NaN,Bachelor's,Employed,20.838977,Rural,...,1.0,5.0,574.5,3.0,2021-10-13 15:21:39.094752,Poor,Yes,Rarely,Apartment,787.0
28,28,43.0,Male,1060.0,Married,NaN,Master's,Self-Employed,41.376716,Suburban,...,0.0,8.0,795.0,7.0,2019-09-06 15:21:39.171102,Average,Yes,Rarely,Condo,699.0
29,29,64.0,Male,131038.0,Single,NaN,PhD,Self-Employed,20.717076,Rural,...,0.0,14.0,495.0,1.0,2020-08-24 15:21:39.155231,Good,Yes,Monthly,House,2360.0
31,31,37.0,Male,4534.0,Divorced,NaN,Bachelor's,Unemployed,26.387481,Suburban,...,0.0,1.0,574.5,1.0,2022-09-21 15:21:39.102694,Poor,No,Daily,House,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199940,1199940,57.0,Female,6871.0,Married,NaN,Bachelor's,Unemployed,24.502801,Rural,...,2.0,19.0,456.0,5.0,2022-01-25 15:21:39.151732,Good,Yes,Rarely,Apartment,1675.0
1199956,1199956,58.0,Male,99860.0,Married,NaN,High School,Self-Employed,14.082348,Urban,...,2.0,18.0,307.0,3.0,2022-05-08 15:21:39.134960,Poor,Yes,Rarely,Condo,2694.0
1199961,1199961,28.0,Female,6868.0,Divorced,NaN,PhD,Employed,18.405534,Urban,...,0.0,17.0,543.0,5.0,2021-12-12 15:21:39.228521,Poor,Yes,Monthly,Apartment,963.0
1199979,1199979,42.0,Male,44691.0,Married,NaN,Bachelor's,Unemployed,24.502801,Suburban,...,0.0,7.0,516.0,8.0,2019-10-10 15:21:39.134960,Poor,Yes,Rarely,Apartment,1358.0


Esta coluna pode ter outliers. Para que não haja influencia maior de poucos valores. É fundamental analisar este item.

In [111]:
# calculate IQR for column Height
Q1 = train['Number of Dependents'].quantile(0.25)
Q3 = train['Number of Dependents'].quantile(0.75)
IQR = Q3 - Q1

# identify outliers
threshold = 1.5
outliers = train[(train['Number of Dependents'] < Q1 - threshold * IQR) | (train['Number of Dependents'] > Q3 + threshold * IQR)]

In [112]:
outliers

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount


Verificamos e não há outliers. Por isto vamos inputar a moda.

In [113]:
np.sort(train['Number of Dependents'].unique())

array([ 0.,  1.,  2.,  3.,  4., nan])

In [114]:
train['Number of Dependents'].mode()[0]

3.0

In [115]:
train['Number of Dependents'].fillna(value=train['Number of Dependents'].mode()[0], inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\3790942853.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Number of Dependents'].fillna(value=train['Number of Dependents'].mode()[0], inplace=True)


In [116]:
test['Number of Dependents'].fillna(value=test['Number of Dependents'].mode()[0], inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\2878746750.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Number of Dependents'].fillna(value=test['Number of Dependents'].mode()[0], inplace=True)


In [117]:
train.isnull().sum()

id                          0
Age                         0
Gender                      0
Annual Income           21252
Marital Status          18529
Number of Dependents        0
Education Level             0
Occupation                  0
Health Score                0
Location                    0
Policy Type                 0
Previous Claims             0
Vehicle Age                 6
Credit Score                0
Insurance Duration          1
Policy Start Date           0
Customer Feedback           0
Smoking Status              0
Exercise Frequency          0
Property Type               0
Premium Amount              0
dtype: int64

In [118]:
train[train['Marital Status'].isnull()]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
45,45,61.0,Male,5095.0,NaN,3.0,Master's,Employed,40.886124,Suburban,...,2.0,7.0,842.0,7.0,2020-08-04 15:21:39.269494,Poor,No,Weekly,House,23.0
132,132,59.0,Male,86031.0,NaN,4.0,Master's,Unemployed,18.878407,Urban,...,0.0,12.0,519.0,8.0,2023-05-16 15:21:39.111551,Average,Yes,Monthly,Condo,2228.0
234,234,59.0,Male,18095.0,NaN,1.0,PhD,Unemployed,23.923010,Rural,...,1.0,17.0,786.0,9.0,2021-08-24 15:21:39.188220,Good,Yes,Rarely,Apartment,1507.0
294,294,26.0,Female,3850.0,NaN,2.0,Bachelor's,Employed,18.417060,Rural,...,0.0,12.0,823.0,1.0,2022-10-15 15:21:39.154008,Poor,No,Daily,Apartment,372.0
304,304,32.0,Female,48913.0,NaN,0.0,Master's,Self-Employed,17.746161,Urban,...,0.0,15.0,560.0,3.0,2024-04-27 15:21:39.255696,Average,No,Rarely,House,2086.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199356,1199356,19.0,Female,1668.0,NaN,3.0,High School,Unemployed,34.079359,Suburban,...,0.0,11.0,300.0,4.0,2022-10-26 15:21:39.126661,Average,Yes,Weekly,Apartment,524.0
1199490,1199490,30.0,Female,4063.0,NaN,2.0,PhD,Unemployed,27.071860,Rural,...,0.0,16.0,494.0,9.0,2024-02-07 15:21:39.107553,Good,No,Weekly,Condo,1770.0
1199497,1199497,18.0,Male,94595.0,NaN,1.0,Master's,Employed,8.080199,Suburban,...,0.0,16.0,821.0,9.0,2022-09-30 15:21:39.288099,Good,No,Daily,Condo,394.0
1199595,1199595,41.0,Female,8074.0,NaN,0.0,PhD,Self-Employed,41.644220,Urban,...,0.0,1.0,514.0,5.0,2024-06-02 15:21:39.130190,Good,No,Daily,House,82.0


In [119]:
train['Marital Status'][train['Number of Dependents'] > 3]

12           Single
17         Divorced
22         Divorced
32         Divorced
35           Single
             ...   
1199982      Single
1199986    Divorced
1199988    Divorced
1199991    Divorced
1199994      Single
Name: Marital Status, Length: 220340, dtype: object

In [120]:
train['Marital Status'].unique()

array(['Married', 'Divorced', 'Single', nan], dtype=object)

In [121]:
train['Marital Status'][train['Number of Dependents'] > 3].mode()[0]

'Single'

In [122]:
train.loc[ ( train['Number of Dependents'] > 3 ) & ( train['Marital Status'].isnull() ), 'Marital Status' ] = train['Marital Status'][train['Number of Dependents'] > 3].mode()[0]

In [123]:
train.loc[ ( train['Number of Dependents'] == 3 ) & ( train['Marital Status'].isnull() ), 'Marital Status' ] = train['Marital Status'][train['Number of Dependents'] == 3].mode()[0]

In [124]:
train.loc[ ( train['Number of Dependents'] == 2 ) & ( train['Marital Status'].isnull() ), 'Marital Status' ] = train['Marital Status'][train['Number of Dependents'] == 2].mode()[0]

In [125]:
train.loc[ ( train['Number of Dependents'] == 1 ) & ( train['Marital Status'].isnull() ), 'Marital Status' ] = train['Marital Status'][train['Number of Dependents'] == 1].mode()[0]

In [126]:
train.loc[ ( train['Number of Dependents'] == 0 ) & ( train['Marital Status'].isnull() ), 'Marital Status' ] = train['Marital Status'][train['Number of Dependents'] == 0].mode()[0]

In [127]:
train['Marital Status'].unique()

array(['Married', 'Divorced', 'Single'], dtype=object)

In [128]:
train[ train['id'] == 132 ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
132,132,59.0,Male,86031.0,Single,4.0,Master's,Unemployed,18.878407,Urban,...,0.0,12.0,519.0,8.0,2023-05-16 15:21:39.111551,Average,Yes,Monthly,Condo,2228.0


In [129]:
train[ train['id'] == 45 ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
45,45,61.0,Male,5095.0,Married,3.0,Master's,Employed,40.886124,Suburban,...,2.0,7.0,842.0,7.0,2020-08-04 15:21:39.269494,Poor,No,Weekly,House,23.0


In [130]:
train[ train['id'] == 294 ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
294,294,26.0,Female,3850.0,Single,2.0,Bachelor's,Employed,18.41706,Rural,...,0.0,12.0,823.0,1.0,2022-10-15 15:21:39.154008,Poor,No,Daily,Apartment,372.0


In [131]:
train[ train['id'] == 234 ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
234,234,59.0,Male,18095.0,Single,1.0,PhD,Unemployed,23.92301,Rural,...,1.0,17.0,786.0,9.0,2021-08-24 15:21:39.188220,Good,Yes,Rarely,Apartment,1507.0


In [132]:
train[ train['id'] == 304 ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
304,304,32.0,Female,48913.0,Single,0.0,Master's,Self-Employed,17.746161,Urban,...,0.0,15.0,560.0,3.0,2024-04-27 15:21:39.255696,Average,No,Rarely,House,2086.0


In [133]:
train.isnull().sum()

id                          0
Age                         0
Gender                      0
Annual Income           21252
Marital Status              0
Number of Dependents        0
Education Level             0
Occupation                  0
Health Score                0
Location                    0
Policy Type                 0
Previous Claims             0
Vehicle Age                 6
Credit Score                0
Insurance Duration          1
Policy Start Date           0
Customer Feedback           0
Smoking Status              0
Exercise Frequency          0
Property Type               0
Premium Amount              0
dtype: int64

In [134]:
train[train['Annual Income'].isnull()]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
36,36,41.0,Female,NaN,Married,3.0,PhD,Self-Employed,14.001630,Urban,...,0.0,15.0,589.0,6.0,2023-03-01 15:21:39.149731,Average,No,Rarely,House,250.0
67,67,45.0,Male,NaN,Married,3.0,High School,Self-Employed,17.988631,Suburban,...,0.0,18.0,375.0,9.0,2022-05-25 15:21:39.237118,Average,Yes,Rarely,House,829.0
89,89,38.0,Female,NaN,Single,3.0,Bachelor's,Employed,40.161460,Suburban,...,1.0,1.0,502.0,4.0,2022-06-25 15:21:39.089116,Poor,Yes,Monthly,Apartment,2728.0
278,278,29.0,Female,NaN,Single,4.0,Master's,Self-Employed,47.839298,Suburban,...,0.0,18.0,416.0,3.0,2020-04-12 15:21:39.282785,Good,Yes,Weekly,House,1162.0
291,291,49.0,Male,NaN,Married,3.0,Bachelor's,Self-Employed,13.723132,Suburban,...,2.0,1.0,762.0,5.0,2023-02-10 15:21:39.099739,Poor,Yes,Rarely,Apartment,385.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199660,1199660,45.0,Male,NaN,Married,1.0,Master's,Employed,34.756945,Urban,...,3.0,9.0,669.0,3.0,2023-02-17 15:21:39.172097,Average,No,Daily,Condo,74.0
1199682,1199682,60.0,Male,NaN,Single,2.0,Bachelor's,Employed,12.976344,Urban,...,0.0,2.0,643.0,8.0,2023-04-10 15:21:39.144722,Poor,No,Daily,House,66.0
1199769,1199769,19.0,Female,NaN,Divorced,0.0,High School,Self-Employed,24.502801,Suburban,...,1.0,4.0,662.0,2.0,2021-12-30 15:21:39.206847,Average,Yes,Monthly,Condo,495.0
1199871,1199871,55.0,Female,NaN,Divorced,3.0,Master's,Self-Employed,19.859580,Suburban,...,0.0,11.0,302.0,4.0,2022-02-19 15:21:39.187211,Poor,No,Rarely,Condo,1377.0


In [135]:
np.sort(train['Annual Income'].unique())

array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 1.49996e+05,
       1.49997e+05,         nan])

In [136]:
train['Annual Income'].median()

23258.0

In [137]:
train['Annual Income'].mean()

32086.923191386115

In [138]:
train['Annual Income'].mode()[0]

0.0

In [139]:
#train['Annual Income'].fillna( value=train['Annual Income'].mode()[0], inplace=True)

In [140]:
train['Annual Income'].fillna( value=-999, inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\1025030485.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Annual Income'].fillna( value=-999, inplace=True)


In [141]:
test['Annual Income'].fillna( value=test['Annual Income'].mode()[0], inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\3299414188.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Annual Income'].fillna( value=test['Annual Income'].mode()[0], inplace=True)


In [142]:
train[ train['id'] == 36 ]

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
36,36,41.0,Female,-999.0,Married,3.0,PhD,Self-Employed,14.00163,Urban,...,0.0,15.0,589.0,6.0,2023-03-01 15:21:39.149731,Average,No,Rarely,House,250.0


In [143]:
train.isnull().sum()

id                      0
Age                     0
Gender                  0
Annual Income           0
Marital Status          0
Number of Dependents    0
Education Level         0
Occupation              0
Health Score            0
Location                0
Policy Type             0
Previous Claims         0
Vehicle Age             6
Credit Score            0
Insurance Duration      1
Policy Start Date       0
Customer Feedback       0
Smoking Status          0
Exercise Frequency      0
Property Type           0
Premium Amount          0
dtype: int64

In [144]:
def removeDataframe(dataframe_name):
    """
    Remove um DataFrame do ambiente global, se ele existir.
    :param dataframe_name: Nome do DataFrame como string (ex: 'df')
    """
    if dataframe_name in globals():
        del globals()[dataframe_name]  # Usando globals() com parênteses
        print(f"O DataFrame '{dataframe_name}' foi deletado.")
    else:
        print(f"O DataFrame '{dataframe_name}' não existe no ambiente.")
        

In [145]:
rows_remove = train[ ( train['Vehicle Age'].isnull() ) | ( train['Insurance Duration'].isnull() ) ]
rows_remove

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
15629,15629,25.0,Female,638.0,Divorced,3.0,PhD,Employed,13.494674,Suburban,...,2.0,NaN,467.0,6.0,2022-06-20 15:21:39.288099,Poor,Yes,Daily,Condo,909.0
53843,53843,36.0,Male,7735.0,Married,3.0,PhD,Employed,10.592719,Suburban,...,0.0,NaN,534.0,2.0,2024-08-08 15:21:39.226954,Average,No,Rarely,Condo,461.0
134847,134847,52.0,Male,20287.0,Divorced,4.0,High School,Unemployed,4.788881,Suburban,...,0.0,NaN,406.0,6.0,2023-10-16 15:21:39.226954,Good,No,Daily,House,2719.0
412847,412847,32.0,Male,1035.0,Single,1.0,PhD,Unemployed,17.491565,Urban,...,2.0,NaN,733.0,3.0,2020-04-24 15:21:39.264504,Average,Yes,Rarely,House,508.0
595207,595207,58.0,Female,-999.0,Married,2.0,Bachelor's,Self-Employed,34.596578,Urban,...,3.0,NaN,311.0,5.0,2022-03-21 15:21:39.271453,Poor,Yes,Monthly,Condo,1628.0
711358,711358,64.0,Male,30206.0,Married,3.0,Master's,Employed,49.551038,Suburban,...,0.0,18.0,581.0,NaN,2022-04-06 15:21:39.203442,Poor,Yes,Rarely,Apartment,1044.0
1068825,1068825,61.0,Male,45279.0,Single,1.0,PhD,Employed,15.268620,Suburban,...,1.0,NaN,622.0,1.0,2020-03-13 15:21:39.274993,Poor,Yes,Rarely,Condo,962.0


In [146]:
#rows_remove_test = test[ ( test['Vehicle Age'].isnull() ) | ( test['Insurance Duration'].isnull() ) ]
#rows_remove_test

In [147]:
rows_mode_test_vehicle_age = test[ test['Vehicle Age'].isnull() ]
rows_mode_test_insurance_duration = test['Insurance Duration'].isnull()

In [148]:
#test['Vehicle Age'].fillna(value=test['Vehicle Age'].mode()[0], inplace=True)

In [149]:
test['Vehicle Age'].fillna(value=-999, inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\1320105162.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Vehicle Age'].fillna(value=-999, inplace=True)


In [150]:
test['Number of Dependents'].fillna(value=test['Insurance Duration'].mode()[0], inplace=True)

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\4218457155.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Number of Dependents'].fillna(value=test['Insurance Duration'].mode()[0], inplace=True)


In [151]:
train = train.drop(rows_remove.index)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,574.5,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,0.0,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,0.0,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,0.0,19.0,574.5,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


In [152]:
#test = test.drop(rows_mode_test_vehicle_age.index)
#test

In [153]:
removeDataframe("rows_remove")

O DataFrame 'rows_remove' foi deletado.


In [154]:
removeDataframe("rows_remove_test")

O DataFrame 'rows_remove_test' não existe no ambiente.


Agora que preenchemos todos os registros nulos. Será tratado as colunas conforme suas definições, utilidades, tipos de dados, faixa de valores para na melhor forma gerar melhores modelos.

In [155]:
train.dtypes

id                        int64
Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age             float64
Credit Score            float64
Insurance Duration      float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
Premium Amount          float64
dtype: object

In [156]:
train['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [157]:
train_gender = pd.get_dummies(train['Gender'], prefix='gender')
train_gender

,gender_Female,gender_Male
0,True,False
1,True,False
2,False,True
3,False,True
4,False,True
...,...,...
1199995,True,False
1199996,False,True
1199997,False,True
1199998,False,True


In [158]:
test_gender = pd.get_dummies(test['Gender'], prefix='gender')
test_gender

,gender_Female,gender_Male
0,True,False
1,True,False
2,True,False
3,True,False
4,False,True
...,...,...
799995,True,False
799996,True,False
799997,True,False
799998,True,False


In [159]:
#continuar daqui
train = pd.concat([train,train_gender], axis=1)

In [160]:
test = pd.concat([test,test_gender], axis=1)

In [161]:
removeDataframe("train_gender")

O DataFrame 'train_gender' foi deletado.


In [162]:
train_marital_status = pd.get_dummies(train['Marital Status'], prefix='marital_status')
train_marital_status

,marital_status_Divorced,marital_status_Married,marital_status_Single
0,False,True,False
1,True,False,False
2,True,False,False
3,False,True,False
4,False,False,True
...,...,...,...
1199995,False,True,False
1199996,True,False,False
1199997,True,False,False
1199998,False,False,True


In [163]:
test_marital_status = pd.get_dummies(test['Marital Status'], prefix='marital_status')
test_marital_status

,marital_status_Divorced,marital_status_Married,marital_status_Single
0,False,False,False
1,False,True,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
799995,False,True,False
799996,False,False,True
799997,False,False,True
799998,False,False,True


In [164]:
train = pd.concat([train,train_marital_status], axis=1)

In [165]:
test = pd.concat([test,test_marital_status], axis=1)

In [166]:
removeDataframe("train_marital_status")

O DataFrame 'train_marital_status' foi deletado.


In [167]:
removeDataframe("test_marital_status")

O DataFrame 'test_marital_status' foi deletado.


In [168]:
#asdasdsad

In [169]:
train['Location'].unique()

array(['Urban', 'Rural', 'Suburban'], dtype=object)

In [170]:
train_location = pd.get_dummies(train['Location'], prefix='location')

In [171]:
test_location = pd.get_dummies(test['Location'], prefix='location')

In [172]:
train = pd.concat([train, train_location], axis=1)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Property Type,Premium Amount,gender_Female,gender_Male,marital_status_Divorced,marital_status_Married,marital_status_Single,location_Rural,location_Suburban,location_Urban
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,House,2869.0,True,False,False,True,False,False,False,True
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,House,1483.0,True,False,True,False,False,True,False,False
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,House,567.0,False,True,True,False,False,False,True,False
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,Apartment,765.0,False,True,False,True,False,True,False,False
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,House,2022.0,False,True,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,Apartment,1303.0,True,False,False,True,False,False,False,True
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,Apartment,821.0,False,True,True,False,False,True,False,False
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,Condo,371.0,False,True,True,False,False,False,True,False
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,Apartment,596.0,False,True,False,False,True,False,True,False


In [173]:
test = pd.concat([test, test_location], axis=1)
test

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Exercise Frequency,Property Type,gender_Female,gender_Male,marital_status_Divorced,marital_status_Married,marital_status_Single,location_Rural,location_Suburban,location_Urban
0,1200000,28.000000,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,Weekly,House,True,False,False,False,False,True,False,False
1,1200001,31.000000,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,Rarely,Apartment,True,False,False,True,False,False,True,False
2,1200002,47.000000,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,Monthly,Condo,True,False,True,False,False,False,False,True
3,1200003,28.000000,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,Daily,House,True,False,True,False,False,False,True,False
4,1200004,24.000000,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,Weekly,House,False,True,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,Female,38782.0,Married,1.0,Bachelor's,Unemployed,14.498639,Rural,...,Daily,Condo,True,False,False,True,False,True,False,False
799996,1999996,24.471782,Female,73462.0,Single,0.0,Master's,Unemployed,8.145748,Rural,...,Daily,Apartment,True,False,False,False,True,True,False,False
799997,1999997,26.000000,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,Monthly,Apartment,True,False,False,False,True,False,False,True
799998,1999998,34.000000,Female,45661.0,Single,3.0,Master's,Unemployed,15.937248,Urban,...,Weekly,Condo,True,False,False,False,True,False,False,True


In [174]:
removeDataframe("train_location")

O DataFrame 'train_location' foi deletado.


In [175]:
removeDataframe("test_location")

O DataFrame 'test_location' foi deletado.


In [176]:
train_occupation = pd.get_dummies(train['Occupation'], prefix='occupation')

In [177]:
test_occupation = pd.get_dummies(test['Occupation'], prefix='occupation')

In [178]:
train = pd.concat([train, train_occupation], axis=1)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,gender_Male,marital_status_Divorced,marital_status_Married,marital_status_Single,location_Rural,location_Suburban,location_Urban,occupation_Employed,occupation_Self-Employed,occupation_Unemployed
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,False,False,True,False,False,False,True,False,True,False
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,False,True,False,False,True,False,False,False,False,True
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,True,True,False,False,False,True,False,False,True,False
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,True,False,True,False,True,False,False,False,False,True
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,True,False,False,True,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,False,False,True,False,False,False,True,False,False,True
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,True,True,False,False,True,False,False,False,True,False
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,True,True,False,False,False,True,False,False,False,True
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,True,False,False,True,False,True,False,False,False,True


In [179]:
test = pd.concat([test, test_occupation], axis=1)
test

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,gender_Male,marital_status_Divorced,marital_status_Married,marital_status_Single,location_Rural,location_Suburban,location_Urban,occupation_Employed,occupation_Self-Employed,occupation_Unemployed
0,1200000,28.000000,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,False,False,False,False,True,False,False,False,True,False
1,1200001,31.000000,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,False,False,True,False,False,True,False,False,True,False
2,1200002,47.000000,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,False,True,False,False,False,False,True,False,False,True
3,1200003,28.000000,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,False,True,False,False,False,True,False,False,True,False
4,1200004,24.000000,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,True,True,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,Female,38782.0,Married,1.0,Bachelor's,Unemployed,14.498639,Rural,...,False,False,True,False,True,False,False,False,False,True
799996,1999996,24.471782,Female,73462.0,Single,0.0,Master's,Unemployed,8.145748,Rural,...,False,False,False,True,True,False,False,False,False,True
799997,1999997,26.000000,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,False,False,False,True,False,False,True,True,False,False
799998,1999998,34.000000,Female,45661.0,Single,3.0,Master's,Unemployed,15.937248,Urban,...,False,False,False,True,False,False,True,False,False,True


In [180]:
removeDataframe("test_occupation")

O DataFrame 'test_occupation' foi deletado.


In [181]:
removeDataframe("train_occupation")

O DataFrame 'train_occupation' foi deletado.


In [182]:
train['Education Level'].unique()

array(["Bachelor's", "Master's", 'High School', 'PhD'], dtype=object)

In [183]:
train_education_level = pd.get_dummies(train['Education Level'], prefix='education_level')

In [184]:
test_education_level = pd.get_dummies(test['Education Level'], prefix='education_level')

In [185]:
train = pd.concat([train, train_education_level], axis=1)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,location_Rural,location_Suburban,location_Urban,occupation_Employed,occupation_Self-Employed,occupation_Unemployed,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_PhD
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,False,False,True,False,True,False,True,False,False,False
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,True,False,False,False,False,True,False,False,True,False
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,False,True,False,False,True,False,False,True,False,False
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,True,False,False,False,False,True,True,False,False,False
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,True,False,False,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,False,False,True,False,False,True,False,False,True,False
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,True,False,False,False,True,False,False,False,True,False
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,False,True,False,False,False,True,False,False,True,False
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,False,True,False,False,False,True,False,False,False,True


In [186]:
test = pd.concat([test, test_education_level], axis=1)
test

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,location_Rural,location_Suburban,location_Urban,occupation_Employed,occupation_Self-Employed,occupation_Unemployed,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_PhD
0,1200000,28.000000,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,True,False,False,False,True,False,True,False,False,False
1,1200001,31.000000,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,False,True,False,False,True,False,False,False,True,False
2,1200002,47.000000,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,False,False,True,False,False,True,False,False,False,True
3,1200003,28.000000,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,False,True,False,False,True,False,False,False,False,True
4,1200004,24.000000,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,False,True,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,Female,38782.0,Married,1.0,Bachelor's,Unemployed,14.498639,Rural,...,True,False,False,False,False,True,True,False,False,False
799996,1999996,24.471782,Female,73462.0,Single,0.0,Master's,Unemployed,8.145748,Rural,...,True,False,False,False,False,True,False,False,True,False
799997,1999997,26.000000,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,False,False,True,True,False,False,False,False,True,False
799998,1999998,34.000000,Female,45661.0,Single,3.0,Master's,Unemployed,15.937248,Urban,...,False,False,True,False,False,True,False,False,True,False


In [187]:
removeDataframe("train_education_level")

O DataFrame 'train_education_level' foi deletado.


In [188]:
removeDataframe("test_education_level")

O DataFrame 'test_education_level' foi deletado.


In [189]:
train['Policy Type'].unique()

array(['Premium', 'Comprehensive', 'Basic'], dtype=object)

In [190]:
train_policy_type = pd.get_dummies(train['Policy Type'], prefix='policy_type')

In [191]:
test_policy_type = pd.get_dummies(test['Policy Type'], prefix='policy_type')

In [192]:
train = pd.concat([train, train_policy_type], axis=1)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,occupation_Employed,occupation_Self-Employed,occupation_Unemployed,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_PhD,policy_type_Basic,policy_type_Comprehensive,policy_type_Premium
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,False,True,False,True,False,False,False,False,False,True
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,False,False,True,False,False,True,False,False,True,False
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,False,True,False,False,True,False,False,False,False,True
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,False,False,True,True,False,False,False,True,False,False
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,False,True,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,False,False,True,False,False,True,False,False,False,True
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,False,True,False,False,False,True,False,False,True,False
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,False,False,True,False,False,True,False,True,False,False
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,False,False,True,False,False,False,True,False,False,True


In [193]:
test = pd.concat([test, test_policy_type], axis=1)
test

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,occupation_Employed,occupation_Self-Employed,occupation_Unemployed,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_PhD,policy_type_Basic,policy_type_Comprehensive,policy_type_Premium
0,1200000,28.000000,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,False,True,False,True,False,False,False,True,False,False
1,1200001,31.000000,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,False,True,False,False,False,True,False,False,False,True
2,1200002,47.000000,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,False,False,True,False,False,False,True,False,True,False
3,1200003,28.000000,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,False,True,False,False,False,False,True,False,True,False
4,1200004,24.000000,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,False,False,True,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,Female,38782.0,Married,1.0,Bachelor's,Unemployed,14.498639,Rural,...,False,False,True,True,False,False,False,False,False,True
799996,1999996,24.471782,Female,73462.0,Single,0.0,Master's,Unemployed,8.145748,Rural,...,False,False,True,False,False,True,False,True,False,False
799997,1999997,26.000000,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,True,False,False,False,False,True,False,False,True,False
799998,1999998,34.000000,Female,45661.0,Single,3.0,Master's,Unemployed,15.937248,Urban,...,False,False,True,False,False,True,False,False,False,True


In [194]:
removeDataframe("train_policy_type")

O DataFrame 'train_policy_type' foi deletado.


In [195]:
removeDataframe("test_policy_type")

O DataFrame 'test_policy_type' foi deletado.


In [196]:
train['Customer Feedback'].unique()

array(['Poor', 'Average', 'Good'], dtype=object)

In [197]:
train_customer_feedback = pd.get_dummies(train['Customer Feedback'], prefix='customer_feedback')

In [198]:
test_customer_feedback = pd.get_dummies(test['Customer Feedback'], prefix='customer_feedback')

In [199]:
train = pd.concat([train, train_customer_feedback], axis=1)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_PhD,policy_type_Basic,policy_type_Comprehensive,policy_type_Premium,customer_feedback_Average,customer_feedback_Good,customer_feedback_Poor
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,True,False,False,False,False,False,True,False,False,True
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,False,False,True,False,False,True,False,True,False,False
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,False,True,False,False,False,False,True,False,True,False
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,True,False,False,False,True,False,False,False,False,True
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,True,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,False,False,True,False,False,False,True,False,False,True
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,False,False,True,False,False,True,False,False,False,True
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,False,False,True,False,True,False,False,False,True,False
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,False,False,False,True,False,False,True,False,False,True


In [200]:
test = pd.concat([test, test_customer_feedback], axis=1)
test

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_PhD,policy_type_Basic,policy_type_Comprehensive,policy_type_Premium,customer_feedback_Average,customer_feedback_Good,customer_feedback_Poor
0,1200000,28.000000,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,True,False,False,False,True,False,False,False,False,True
1,1200001,31.000000,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,False,False,True,False,False,False,True,False,True,False
2,1200002,47.000000,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,False,False,False,True,False,True,False,True,False,False
3,1200003,28.000000,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,False,False,False,True,False,True,False,False,False,True
4,1200004,24.000000,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,False,True,False,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,Female,38782.0,Married,1.0,Bachelor's,Unemployed,14.498639,Rural,...,True,False,False,False,False,False,True,True,False,False
799996,1999996,24.471782,Female,73462.0,Single,0.0,Master's,Unemployed,8.145748,Rural,...,False,False,True,False,True,False,False,False,True,False
799997,1999997,26.000000,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,False,False,True,False,False,True,False,False,False,True
799998,1999998,34.000000,Female,45661.0,Single,3.0,Master's,Unemployed,15.937248,Urban,...,False,False,True,False,False,False,True,True,False,False


In [201]:
removeDataframe("train_customer_feedback")

O DataFrame 'train_customer_feedback' foi deletado.


In [202]:
removeDataframe("test_customer_feedback")

O DataFrame 'test_customer_feedback' foi deletado.


In [203]:
train['Smoking Status'].unique()

array(['No', 'Yes'], dtype=object)

In [204]:
train_smoking_status = pd.get_dummies(train['Smoking Status'], prefix='smoking_status')

In [205]:
test_smoking_status = pd.get_dummies(test['Smoking Status'], prefix='smoking_status')

In [206]:
train = pd.concat([train, train_smoking_status], axis=1)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,education_level_Master's,education_level_PhD,policy_type_Basic,policy_type_Comprehensive,policy_type_Premium,customer_feedback_Average,customer_feedback_Good,customer_feedback_Poor,smoking_status_No,smoking_status_Yes
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,False,False,False,False,True,False,False,True,True,False
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,True,False,False,True,False,True,False,False,False,True
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,False,False,False,False,True,False,True,False,False,True
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,False,False,True,False,False,False,False,True,False,True
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,False,False,False,False,True,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,True,False,False,False,True,False,False,True,True,False
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,True,False,False,True,False,False,False,True,True,False
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,True,False,True,False,False,False,True,False,True,False
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,False,True,False,False,True,False,False,True,True,False


In [207]:
test = pd.concat([test, test_smoking_status], axis=1)
test

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,education_level_Master's,education_level_PhD,policy_type_Basic,policy_type_Comprehensive,policy_type_Premium,customer_feedback_Average,customer_feedback_Good,customer_feedback_Poor,smoking_status_No,smoking_status_Yes
0,1200000,28.000000,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,False,False,True,False,False,False,False,True,False,True
1,1200001,31.000000,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,True,False,False,False,True,False,True,False,False,True
2,1200002,47.000000,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,False,True,False,True,False,True,False,False,False,True
3,1200003,28.000000,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,False,True,False,True,False,False,False,True,False,True
4,1200004,24.000000,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,False,False,False,False,True,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,Female,38782.0,Married,1.0,Bachelor's,Unemployed,14.498639,Rural,...,False,False,False,False,True,True,False,False,False,True
799996,1999996,24.471782,Female,73462.0,Single,0.0,Master's,Unemployed,8.145748,Rural,...,True,False,True,False,False,False,True,False,True,False
799997,1999997,26.000000,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,True,False,False,True,False,False,False,True,True,False
799998,1999998,34.000000,Female,45661.0,Single,3.0,Master's,Unemployed,15.937248,Urban,...,True,False,False,False,True,True,False,False,True,False


In [208]:
removeDataframe("train_smoking_status")

O DataFrame 'train_smoking_status' foi deletado.


In [209]:
removeDataframe("test_smoking_status")

O DataFrame 'test_smoking_status' foi deletado.


In [210]:
train['Exercise Frequency'].unique()

array(['Weekly', 'Monthly', 'Daily', 'Rarely'], dtype=object)

In [211]:
train_exercise_frequency = pd.get_dummies(train['Exercise Frequency'], prefix='exercise_frequency')

In [212]:
test_exercise_frequency = pd.get_dummies(test['Exercise Frequency'], prefix='exercise_frequency')

In [213]:
train = pd.concat([train, train_exercise_frequency], axis=1)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,policy_type_Premium,customer_feedback_Average,customer_feedback_Good,customer_feedback_Poor,smoking_status_No,smoking_status_Yes,exercise_frequency_Daily,exercise_frequency_Monthly,exercise_frequency_Rarely,exercise_frequency_Weekly
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,True,False,False,True,True,False,False,False,False,True
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,False,True,False,False,False,True,False,True,False,False
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,True,False,True,False,False,True,False,False,False,True
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,False,False,False,True,False,True,True,False,False,False
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,True,False,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,True,False,False,True,True,False,True,False,False,False
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,False,False,False,True,True,False,False,False,False,True
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,False,False,True,False,True,False,False,True,False,False
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,True,False,False,True,True,False,True,False,False,False


In [214]:
test = pd.concat([test, test_exercise_frequency], axis=1)
test

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,policy_type_Premium,customer_feedback_Average,customer_feedback_Good,customer_feedback_Poor,smoking_status_No,smoking_status_Yes,exercise_frequency_Daily,exercise_frequency_Monthly,exercise_frequency_Rarely,exercise_frequency_Weekly
0,1200000,28.000000,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,False,False,False,True,False,True,False,False,False,True
1,1200001,31.000000,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,True,False,True,False,False,True,False,False,True,False
2,1200002,47.000000,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,False,True,False,False,False,True,False,True,False,False
3,1200003,28.000000,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,False,False,False,True,False,True,True,False,False,False
4,1200004,24.000000,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,True,True,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,Female,38782.0,Married,1.0,Bachelor's,Unemployed,14.498639,Rural,...,True,True,False,False,False,True,True,False,False,False
799996,1999996,24.471782,Female,73462.0,Single,0.0,Master's,Unemployed,8.145748,Rural,...,False,False,True,False,True,False,True,False,False,False
799997,1999997,26.000000,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,False,False,False,True,True,False,False,True,False,False
799998,1999998,34.000000,Female,45661.0,Single,3.0,Master's,Unemployed,15.937248,Urban,...,True,True,False,False,True,False,False,False,False,True


In [215]:
removeDataframe("train_exercise_frequency")

O DataFrame 'train_exercise_frequency' foi deletado.


In [216]:
removeDataframe("test_exercise_frequency")

O DataFrame 'test_exercise_frequency' foi deletado.


In [217]:
train['Property Type'].unique()

array(['House', 'Apartment', 'Condo'], dtype=object)

In [218]:
train_property_type = pd.get_dummies(train['Property Type'], prefix='property_type')

In [219]:
test_property_type = pd.get_dummies(test['Property Type'], prefix='property_type')

In [220]:
train = pd.concat([train, train_property_type], axis=1)
train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,customer_feedback_Poor,smoking_status_No,smoking_status_Yes,exercise_frequency_Daily,exercise_frequency_Monthly,exercise_frequency_Rarely,exercise_frequency_Weekly,property_type_Apartment,property_type_Condo,property_type_House
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,True,True,False,False,False,False,True,False,False,True
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,...,False,False,True,False,True,False,False,False,False,True
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,False,False,True,False,False,False,True,False,False,True
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,Unemployed,10.938144,Rural,...,True,False,True,True,False,False,False,True,False,False
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,True,False,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,True,True,False,True,False,False,False,True,False,False
1199996,1199996,54.0,Male,35786.0,Divorced,3.0,Master's,Self-Employed,11.483482,Rural,...,True,True,False,False,False,False,True,True,False,False
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,Unemployed,14.724469,Suburban,...,False,True,False,False,True,False,False,False,True,False
1199998,1199998,55.0,Male,0.0,Single,1.0,PhD,Unemployed,18.547381,Suburban,...,True,True,False,True,False,False,False,True,False,False


In [221]:
test = pd.concat([test, test_property_type], axis=1)
test

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,customer_feedback_Poor,smoking_status_No,smoking_status_Yes,exercise_frequency_Daily,exercise_frequency_Monthly,exercise_frequency_Rarely,exercise_frequency_Weekly,property_type_Apartment,property_type_Condo,property_type_House
0,1200000,28.000000,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,...,True,False,True,False,False,False,True,False,False,True
1,1200001,31.000000,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,...,False,False,True,False,False,True,False,True,False,False
2,1200002,47.000000,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,...,False,False,True,False,True,False,False,False,True,False
3,1200003,28.000000,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,...,True,False,True,True,False,False,False,False,False,True
4,1200004,24.000000,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,...,False,True,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,Female,38782.0,Married,1.0,Bachelor's,Unemployed,14.498639,Rural,...,False,False,True,True,False,False,False,False,True,False
799996,1999996,24.471782,Female,73462.0,Single,0.0,Master's,Unemployed,8.145748,Rural,...,False,True,False,True,False,False,False,True,False,False
799997,1999997,26.000000,Female,35178.0,Single,0.0,Master's,Employed,6.636583,Urban,...,True,True,False,False,True,False,False,True,False,False
799998,1999998,34.000000,Female,45661.0,Single,3.0,Master's,Unemployed,15.937248,Urban,...,False,True,False,False,False,False,True,False,True,False


In [222]:
removeDataframe("train_property_type")

O DataFrame 'train_property_type' foi deletado.


In [223]:
removeDataframe("test_property_type")

O DataFrame 'test_property_type' foi deletado.


In [224]:
#Remover as colunas originais substituídas pelas novas features.
columns_droped = ['Gender','Marital Status','Location', 'Occupation', 'Education Level', 
            'Policy Type', 'Customer Feedback', 'Smoking Status','Exercise Frequency','Property Type',
                  #'Policy Start Date'
           ]

In [225]:
train = train.drop(columns_droped, axis='columns')

In [226]:
test = test.drop(columns_droped, axis='columns')

In [227]:
#remover variáveis temporárias

#rows_remove.drop()
#train_gender.drop()
#train_marital_status.drop()
#train_location.drop()
#train_occupation.drop()
#train_education_level.drop()
#train_policy_type.drop()
#train_customer_feedback.drop()
#train_smoking_status.drop()
#train_exercise_frequency.drop()
#train_property_type.drop()

test

In [228]:
train.dtypes

id                               int64
Age                            float64
Annual Income                  float64
Number of Dependents           float64
Health Score                   float64
Previous Claims                float64
Vehicle Age                    float64
Credit Score                   float64
Insurance Duration             float64
Policy Start Date               object
Premium Amount                 float64
gender_Female                     bool
gender_Male                       bool
marital_status_Divorced           bool
marital_status_Married            bool
marital_status_Single             bool
location_Rural                    bool
location_Suburban                 bool
location_Urban                    bool
occupation_Employed               bool
occupation_Self-Employed          bool
occupation_Unemployed             bool
education_level_Bachelor's        bool
education_level_High School       bool
education_level_Master's          bool
education_level_PhD      

In [229]:
train['id'] = train['id'].astype('int32')

In [230]:
test['id'] = test['id'].astype('int32')

In [231]:
train['id']

0                0
1                1
2                2
3                3
4                4
            ...   
1199995    1199995
1199996    1199996
1199997    1199997
1199998    1199998
1199999    1199999
Name: id, Length: 1199993, dtype: int32

In [232]:
columns = [col for col in train.columns if col != 'Premium Amount']  # Todas as colunas exceto 'Premium Amount'
columns.append('Premium Amount')  # Adiciona 'Premium Amount' no final
train = train[columns]  # Reordena o DataFrame

In [233]:
train.columns

Index(['id', 'Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Policy Start Date', 'gender_Female', 'gender_Male',
       'marital_status_Divorced', 'marital_status_Married',
       'marital_status_Single', 'location_Rural', 'location_Suburban',
       'location_Urban', 'occupation_Employed', 'occupation_Self-Employed',
       'occupation_Unemployed', 'education_level_Bachelor's',
       'education_level_High School', 'education_level_Master's',
       'education_level_PhD', 'policy_type_Basic', 'policy_type_Comprehensive',
       'policy_type_Premium', 'customer_feedback_Average',
       'customer_feedback_Good', 'customer_feedback_Poor', 'smoking_status_No',
       'smoking_status_Yes', 'exercise_frequency_Daily',
       'exercise_frequency_Monthly', 'exercise_frequency_Rarely',
       'exercise_frequency_Weekly', 'property_type_Apartment',
       'property_type_Condo', 'property_type

In [234]:
test = test[columns[0:-1]]  # Reordena o DataFrame
test

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,...,customer_feedback_Poor,smoking_status_No,smoking_status_Yes,exercise_frequency_Daily,exercise_frequency_Monthly,exercise_frequency_Rarely,exercise_frequency_Weekly,property_type_Apartment,property_type_Condo,property_type_House
0,1200000,28.000000,2310.0,4.0,7.657981,0.0,19.0,300.0,1.0,2023-06-04 15:21:39.245086,...,True,False,True,False,False,False,True,False,False,True
1,1200001,31.000000,126031.0,2.0,13.381379,0.0,14.0,372.0,8.0,2024-04-22 15:21:39.224915,...,False,False,True,False,False,True,False,True,False,False
2,1200002,47.000000,17092.0,0.0,24.354527,0.0,16.0,819.0,9.0,2023-04-05 15:21:39.134960,...,False,False,True,False,True,False,False,False,True,False
3,1200003,28.000000,30424.0,3.0,5.136225,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,...,True,False,True,True,False,False,False,False,False,True
4,1200004,24.000000,10863.0,2.0,11.844155,0.0,14.0,755.0,7.0,2021-11-26 15:21:39.259788,...,False,True,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1999995,50.000000,38782.0,1.0,14.498639,0.0,8.0,309.0,2.0,2021-07-09 15:21:39.184157,...,False,False,True,True,False,False,False,False,True,False
799996,1999996,24.471782,73462.0,0.0,8.145748,2.0,0.0,300.0,2.0,2023-03-28 15:21:39.250151,...,False,True,False,True,False,False,False,True,False,False
799997,1999997,26.000000,35178.0,0.0,6.636583,0.0,10.0,300.0,6.0,2019-09-30 15:21:39.132191,...,True,True,False,False,True,False,False,True,False,False
799998,1999998,34.000000,45661.0,3.0,15.937248,2.0,17.0,467.0,7.0,2022-05-09 15:21:39.253660,...,False,True,False,False,False,False,True,False,True,False


In [235]:
train.tail()

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,...,smoking_status_No,smoking_status_Yes,exercise_frequency_Daily,exercise_frequency_Monthly,exercise_frequency_Rarely,exercise_frequency_Weekly,property_type_Apartment,property_type_Condo,property_type_House,Premium Amount
1199995,1199995,36.0,27316.0,0.0,13.772907,0.0,5.0,372.0,3.0,2023-05-03 15:21:39.257696,...,True,False,True,False,False,False,True,False,False,1303.0
1199996,1199996,54.0,35786.0,3.0,11.483482,0.0,10.0,597.0,4.0,2022-09-10 15:21:39.134960,...,True,False,False,False,False,True,True,False,False,821.0
1199997,1199997,19.0,51884.0,0.0,14.724469,0.0,19.0,574.5,6.0,2021-05-25 15:21:39.106582,...,True,False,False,True,False,False,False,True,False,371.0
1199998,1199998,55.0,0.0,1.0,18.547381,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,...,True,False,True,False,False,False,True,False,False,596.0
1199999,1199999,21.0,0.0,0.0,10.125323,0.0,18.0,502.0,6.0,2020-08-26 15:21:39.155231,...,False,True,False,True,False,False,False,False,True,2480.0


In [236]:
train.dtypes

id                               int32
Age                            float64
Annual Income                  float64
Number of Dependents           float64
Health Score                   float64
Previous Claims                float64
Vehicle Age                    float64
Credit Score                   float64
Insurance Duration             float64
Policy Start Date               object
gender_Female                     bool
gender_Male                       bool
marital_status_Divorced           bool
marital_status_Married            bool
marital_status_Single             bool
location_Rural                    bool
location_Suburban                 bool
location_Urban                    bool
occupation_Employed               bool
occupation_Self-Employed          bool
occupation_Unemployed             bool
education_level_Bachelor's        bool
education_level_High School       bool
education_level_Master's          bool
education_level_PhD               bool
policy_type_Basic        

In [237]:
train

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,...,smoking_status_No,smoking_status_Yes,exercise_frequency_Daily,exercise_frequency_Monthly,exercise_frequency_Rarely,exercise_frequency_Weekly,property_type_Apartment,property_type_Condo,property_type_House,Premium Amount
0,0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,...,True,False,False,False,False,True,False,False,True,2869.0
1,1,39.0,31678.0,3.0,15.569731,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,...,False,True,False,True,False,False,False,False,True,1483.0
2,2,23.0,25602.0,3.0,47.177549,1.0,14.0,574.5,3.0,2023-09-30 15:21:39.221386,...,False,True,False,False,False,True,False,False,True,567.0
3,3,21.0,141855.0,2.0,10.938144,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,...,False,True,True,False,False,False,True,False,False,765.0
4,4,21.0,39651.0,1.0,20.376094,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,...,False,True,False,False,False,True,False,False,True,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,27316.0,0.0,13.772907,0.0,5.0,372.0,3.0,2023-05-03 15:21:39.257696,...,True,False,True,False,False,False,True,False,False,1303.0
1199996,1199996,54.0,35786.0,3.0,11.483482,0.0,10.0,597.0,4.0,2022-09-10 15:21:39.134960,...,True,False,False,False,False,True,True,False,False,821.0
1199997,1199997,19.0,51884.0,0.0,14.724469,0.0,19.0,574.5,6.0,2021-05-25 15:21:39.106582,...,True,False,False,True,False,False,False,True,False,371.0
1199998,1199998,55.0,0.0,1.0,18.547381,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,...,True,False,True,False,False,False,True,False,False,596.0


In [238]:
def create_policy_start_date_separated_features(df):
    df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
    df['Policy_Start_Year'] = df['Policy Start Date'].dt.year
    df['Policy_Start_Month'] = df['Policy Start Date'].dt.month
    df['Policy_Start_Day'] = df['Policy Start Date'].dt.day
    

    return df

In [239]:
def circular_features_time(df, date_column='Policy Start Date'):
    df[date_column] = pd.to_datetime(df[date_column])
    
    df['day_of_week'] = df[date_column].dt.dayofweek
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    df['day_of_year'] = df[date_column].dt.dayofyear
    df['year_day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['year_day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    
    df['month'] = df[date_column].dt.month
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    df["seconds_since_1970"] = df[date_column].astype("int64") // 10**9
    df = df.drop('Policy Start Date', axis=1)

    return df

In [240]:
train = create_policy_start_date_separated_features(train)
test = create_policy_start_date_separated_features(test)

train = circular_features_time(train)
test = circular_features_time(test)

In [241]:
train

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,gender_Female,...,day_of_week,day_sin,day_cos,day_of_year,year_day_sin,year_day_cos,month,month_sin,month_cos,seconds_since_1970
0,0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.0,5.0,True,...,5,-0.974928,-0.222521,357,-0.137279,0.990532,12,-2.449294e-16,1.000000e+00,1703344899
1,1,39.0,31678.0,3.0,15.569731,1.0,12.0,694.0,2.0,True,...,0,0.000000,1.000000,163,0.329408,-0.944188,6,1.224647e-16,-1.000000e+00,1686583299
2,2,23.0,25602.0,3.0,47.177549,1.0,14.0,574.5,3.0,False,...,5,-0.974928,-0.222521,273,-0.999917,-0.012910,9,-1.000000e+00,-1.836970e-16,1696087299
3,3,21.0,141855.0,2.0,10.938144,1.0,0.0,367.0,1.0,False,...,2,0.974928,-0.222521,164,0.313107,-0.949718,6,1.224647e-16,-1.000000e+00,1718205699
4,4,21.0,39651.0,1.0,20.376094,0.0,8.0,598.0,4.0,False,...,2,0.974928,-0.222521,335,-0.493776,0.869589,12,-2.449294e-16,1.000000e+00,1638372099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,27316.0,0.0,13.772907,0.0,5.0,372.0,3.0,True,...,2,0.974928,-0.222521,123,0.854322,-0.519744,5,5.000000e-01,-8.660254e-01,1683127299
1199996,1199996,54.0,35786.0,3.0,11.483482,0.0,10.0,597.0,4.0,False,...,5,-0.974928,-0.222521,253,-0.936881,-0.349647,9,-1.000000e+00,-1.836970e-16,1662823299
1199997,1199997,19.0,51884.0,0.0,14.724469,0.0,19.0,574.5,6.0,False,...,1,0.781831,0.623490,145,0.601624,-0.798779,5,5.000000e-01,-8.660254e-01,1621956099
1199998,1199998,55.0,0.0,1.0,18.547381,1.0,7.0,407.0,4.0,False,...,6,-0.781831,0.623490,262,-0.979614,-0.200891,9,-1.000000e+00,-1.836970e-16,1632064899


In [242]:
train['gender_Female'].unique()

array([ True, False])

In [243]:
train['gender_Female'].astype('int16')

0          1
1          1
2          0
3          0
4          0
          ..
1199995    1
1199996    0
1199997    0
1199998    0
1199999    1
Name: gender_Female, Length: 1199993, dtype: int16

In [244]:
test['gender_Female'].astype('int16')

0         1
1         1
2         1
3         1
4         0
         ..
799995    1
799996    1
799997    1
799998    1
799999    0
Name: gender_Female, Length: 800000, dtype: int16

In [245]:
train['Age'] = train['Age'].astype('int16')
train['Annual Income'] = train['Annual Income'].astype('int16')
train['Number of Dependents'] = train['Number of Dependents'].astype('int16')
train['Health Score'] = train['Health Score'].astype('float32')
train['Previous Claims'] = train['Previous Claims'].astype('int16')
train['Vehicle Age'] = train['Vehicle Age'].astype('int16')
train['Credit Score'] = train['Credit Score'].astype('int16')
train['Insurance Duration'] = train['Insurance Duration'].astype('int16')

In [246]:
test.isnull().sum()

id                             0
Age                            0
Annual Income                  0
Number of Dependents           0
Health Score                   0
Previous Claims                0
Vehicle Age                    0
Credit Score                   0
Insurance Duration             2
gender_Female                  0
gender_Male                    0
marital_status_Divorced        0
marital_status_Married         0
marital_status_Single          0
location_Rural                 0
location_Suburban              0
location_Urban                 0
occupation_Employed            0
occupation_Self-Employed       0
occupation_Unemployed          0
education_level_Bachelor's     0
education_level_High School    0
education_level_Master's       0
education_level_PhD            0
policy_type_Basic              0
policy_type_Comprehensive      0
policy_type_Premium            0
customer_feedback_Average      0
customer_feedback_Good         0
customer_feedback_Poor         0
smoking_st

In [247]:
test.isna().sum()

id                             0
Age                            0
Annual Income                  0
Number of Dependents           0
Health Score                   0
Previous Claims                0
Vehicle Age                    0
Credit Score                   0
Insurance Duration             2
gender_Female                  0
gender_Male                    0
marital_status_Divorced        0
marital_status_Married         0
marital_status_Single          0
location_Rural                 0
location_Suburban              0
location_Urban                 0
occupation_Employed            0
occupation_Self-Employed       0
occupation_Unemployed          0
education_level_Bachelor's     0
education_level_High School    0
education_level_Master's       0
education_level_PhD            0
policy_type_Basic              0
policy_type_Comprehensive      0
policy_type_Premium            0
customer_feedback_Average      0
customer_feedback_Good         0
customer_feedback_Poor         0
smoking_st

In [248]:
test[ (test['Insurance Duration'] == 0) | (test['Insurance Duration'].isna()) | (test['Insurance Duration'].isnull()) ]

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,gender_Female,...,day_of_week,day_sin,day_cos,day_of_year,year_day_sin,year_day_cos,month,month_sin,month_cos,seconds_since_1970
627445,1827445,53.0,5933.0,3.0,25.162516,0.0,0.0,673.0,NaN,False,...,4,-0.433884,-0.900969,41,0.648630,0.761104,2,0.866025,0.500000,1676042499
706315,1906315,23.0,35357.0,1.0,19.908571,2.0,14.0,481.0,NaN,False,...,5,-0.974928,-0.222521,25,0.417194,0.908818,1,0.500000,0.866025,1579965699


In [249]:
#test['Insurance Duration'][ (test['Insurance Duration'] == 0) | (test['Insurance Duration'].isna()) | (test['Insurance Duration'].isnull()) ] = test['Insurance Duration'].mode()[0]

In [250]:
test['Insurance Duration'][ (test['Insurance Duration'] == 0) | (test['Insurance Duration'].isna()) | (test['Insurance Duration'].isnull()) ] = -999

C:\Users\makrr\AppData\Local\Temp\ipykernel_12704\3811613336.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test['Insurance Duration'][ (test['Insurance Duration'] == 0) | (test['Insurance Duration'].isna()) | (test['Insurance Duration'

In [251]:
test[ (test['Insurance Duration'] == 0) | (test['Insurance Duration'].isna()) | (test['Insurance Duration'].isnull()) ]

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,gender_Female,...,day_of_week,day_sin,day_cos,day_of_year,year_day_sin,year_day_cos,month,month_sin,month_cos,seconds_since_1970


In [252]:
test['Age'] = test['Age'].astype('int16')
test['Annual Income'] = test['Annual Income'].astype('int16')
test['Number of Dependents'] = test['Number of Dependents'].astype('int16')
test['Health Score'] = test['Health Score'].astype('float32')
test['Previous Claims'] = test['Previous Claims'].astype('int16')
test['Vehicle Age'] = test['Vehicle Age'].astype('int16')
test['Credit Score'] = test['Credit Score'].astype('int16')
test['Insurance Duration'] = test['Insurance Duration'].astype('int16')

In [253]:
train['gender_Female'] = train['gender_Female'].astype('int16')
train['smoking_status_No'] = train['smoking_status_No'].astype('int16')
train['smoking_status_Yes'] = train['smoking_status_Yes'].astype('int16')
train['exercise_frequency_Daily'] = train['exercise_frequency_Daily'].astype('int16')
train['exercise_frequency_Monthly'] = train['exercise_frequency_Monthly'].astype('int16')
train['exercise_frequency_Rarely'] = train['exercise_frequency_Rarely'].astype('int16')
train['exercise_frequency_Weekly'] = train['exercise_frequency_Weekly'].astype('int16')
train['smoking_status_No'] = train['smoking_status_No'].astype('int16')
train['property_type_Apartment'] = train['property_type_Apartment'].astype('int16')
train['property_type_Condo'] = train['property_type_Condo'].astype('int16')
train['property_type_House'] = train['property_type_House'].astype('int16')

In [254]:
test['gender_Female'] = test['gender_Female'].astype('int16')
test['smoking_status_No'] = test['smoking_status_No'].astype('int16')
test['smoking_status_Yes'] = test['smoking_status_Yes'].astype('int16')
test['exercise_frequency_Daily'] = test['exercise_frequency_Daily'].astype('int16')
test['exercise_frequency_Monthly'] = test['exercise_frequency_Monthly'].astype('int16')
test['exercise_frequency_Rarely'] = test['exercise_frequency_Rarely'].astype('int16')
test['exercise_frequency_Weekly'] = test['exercise_frequency_Weekly'].astype('int16')
test['smoking_status_No'] = test['smoking_status_No'].astype('int16')
test['property_type_Apartment'] = test['property_type_Apartment'].astype('int16')
test['property_type_Condo'] = test['property_type_Condo'].astype('int16')
test['property_type_House'] = test['property_type_House'].astype('int16')

In [255]:
train['gender_Male'] = train['gender_Male'].astype('int16')
train['marital_status_Divorced'] = train['marital_status_Divorced'].astype('int16')
train['marital_status_Married'] = train['marital_status_Married'].astype('int16')
train['marital_status_Single'] = train['marital_status_Single'].astype('int16')
train['location_Rural'] = train['location_Rural'].astype('int16')
train['location_Suburban'] = train['location_Suburban'].astype('int16')
train['location_Urban'] = train['location_Urban'].astype('int16')
train['occupation_Employed'] = train['occupation_Employed'].astype('int16')
train['occupation_Self-Employed'] = train['occupation_Self-Employed'].astype('int16')
train['occupation_Unemployed'] = train['occupation_Unemployed'].astype('int16')

In [256]:
test['gender_Male'] = test['gender_Male'].astype('int16')
test['marital_status_Divorced'] = test['marital_status_Divorced'].astype('int16')
test['marital_status_Married'] = test['marital_status_Married'].astype('int16')
test['marital_status_Single'] = test['marital_status_Single'].astype('int16')
test['location_Rural'] = test['location_Rural'].astype('int16')
test['location_Suburban'] = test['location_Suburban'].astype('int16')
test['location_Urban'] = test['location_Urban'].astype('int16')
test['occupation_Employed'] = test['occupation_Employed'].astype('int16')
test['occupation_Self-Employed'] = test['occupation_Self-Employed'].astype('int16')
test['occupation_Unemployed'] = test['occupation_Unemployed'].astype('int16')

In [257]:
train["education_level_Bachelor's"] = train["education_level_Bachelor's"].astype('int16')
train['education_level_High School'] = train['education_level_High School'].astype('int16')
train["education_level_Master's"] = train["education_level_Master's"].astype('int16')
train['education_level_PhD'] = train['education_level_PhD'].astype('int16')
train['policy_type_Basic'] = train['policy_type_Basic'].astype('int16')
train['policy_type_Comprehensive'] = train['policy_type_Comprehensive'].astype('int16')
train['policy_type_Premium'] = train['policy_type_Premium'].astype('int16')
train['customer_feedback_Average'] = train['customer_feedback_Average'].astype('int16')
train['customer_feedback_Good'] = train['customer_feedback_Good'].astype('int16')
train['customer_feedback_Poor'] = train['customer_feedback_Poor'].astype('int16')

In [258]:
test["education_level_Bachelor's"] = test["education_level_Bachelor's"].astype('int16')
test['education_level_High School'] = test['education_level_High School'].astype('int16')
test["education_level_Master's"] = test["education_level_Master's"].astype('int16')
test['education_level_PhD'] = test['education_level_PhD'].astype('int16')
test['policy_type_Basic'] = test['policy_type_Basic'].astype('int16')
test['policy_type_Comprehensive'] = test['policy_type_Comprehensive'].astype('int16')
test['policy_type_Premium'] = test['policy_type_Premium'].astype('int16')
test['customer_feedback_Average'] = test['customer_feedback_Average'].astype('int16')
test['customer_feedback_Good'] = test['customer_feedback_Good'].astype('int16')
test['customer_feedback_Poor'] = test['customer_feedback_Poor'].astype('int16')

In [259]:
#train = train[:1000]
train

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,gender_Female,...,day_of_week,day_sin,day_cos,day_of_year,year_day_sin,year_day_cos,month,month_sin,month_cos,seconds_since_1970
0,0,19,10049,1,22.598761,2,17,372,5,1,...,5,-0.974928,-0.222521,357,-0.137279,0.990532,12,-2.449294e-16,1.000000e+00,1703344899
1,1,39,31678,3,15.569731,1,12,694,2,1,...,0,0.000000,1.000000,163,0.329408,-0.944188,6,1.224647e-16,-1.000000e+00,1686583299
2,2,23,25602,3,47.177547,1,14,574,3,0,...,5,-0.974928,-0.222521,273,-0.999917,-0.012910,9,-1.000000e+00,-1.836970e-16,1696087299
3,3,21,10783,2,10.938144,1,0,367,1,0,...,2,0.974928,-0.222521,164,0.313107,-0.949718,6,1.224647e-16,-1.000000e+00,1718205699
4,4,21,-25885,1,20.376093,0,8,598,4,0,...,2,0.974928,-0.222521,335,-0.493776,0.869589,12,-2.449294e-16,1.000000e+00,1638372099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36,27316,0,13.772907,0,5,372,3,1,...,2,0.974928,-0.222521,123,0.854322,-0.519744,5,5.000000e-01,-8.660254e-01,1683127299
1199996,1199996,54,-29750,3,11.483482,0,10,597,4,0,...,5,-0.974928,-0.222521,253,-0.936881,-0.349647,9,-1.000000e+00,-1.836970e-16,1662823299
1199997,1199997,19,-13652,0,14.724468,0,19,574,6,0,...,1,0.781831,0.623490,145,0.601624,-0.798779,5,5.000000e-01,-8.660254e-01,1621956099
1199998,1199998,55,0,1,18.547380,1,7,407,4,0,...,6,-0.781831,0.623490,262,-0.979614,-0.200891,9,-1.000000e+00,-1.836970e-16,1632064899


In [260]:
#test = test[:1000]
#test

In [261]:
#Vamos averiguar se há algum registro nulo para tratar
train.isnull().sum()

id                             0
Age                            0
Annual Income                  0
Number of Dependents           0
Health Score                   0
Previous Claims                0
Vehicle Age                    0
Credit Score                   0
Insurance Duration             0
gender_Female                  0
gender_Male                    0
marital_status_Divorced        0
marital_status_Married         0
marital_status_Single          0
location_Rural                 0
location_Suburban              0
location_Urban                 0
occupation_Employed            0
occupation_Self-Employed       0
occupation_Unemployed          0
education_level_Bachelor's     0
education_level_High School    0
education_level_Master's       0
education_level_PhD            0
policy_type_Basic              0
policy_type_Comprehensive      0
policy_type_Premium            0
customer_feedback_Average      0
customer_feedback_Good         0
customer_feedback_Poor         0
smoking_st

In [262]:
# 'gender_Female','gender_Male','marital_status_Divorced', 
#     'marital_status_Married','marital_status_Single',
#     'marital_status_Divorced',
# colunas_remover = [
#     'location_Rural','location_Suburban',
#     'location_Urban','exercise_frequency_Daily','exercise_frequency_Monthly',
#     'exercise_frequency_Rarely','exercise_frequency_Weekly','Number of Dependents'
# ]

In [263]:
# train = train.drop(colunas_remover, axis=1)
# train

In [264]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(train)

In [ ]:
sns.heatmap(data_scaled)

In [ ]:
pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(data_scaled)
pca_transformed

In [ ]:
sns.heatmap(pca_transformed)

In [ ]:
#sns.pairplot(train.corr(), diag_kind='kde')

In [ ]:
#total_base = sns.pairplot(train, diag_kind='kde')

In [ ]:
#total_base

In [ ]:
#fig = total_base

In [ ]:
# Salva o gráfico em um arquivo
# fig.savefig("pairplot.png")  # Salva como arquivo PNG

# Mostra uma mensagem confirmando
# print("Gráfico salvo como 'pairplot.png'")

In [ ]:
def RMSLE(true,pred):
    true_log = np.log1p(true)
    pred_log = np.log1p(pred)
    m = np.sqrt(np.mean( (true_log-pred_log)**2.0 ))
    return m

In [ ]:
pred = np.exp( np.mean( np.log1p(train["Premium Amount"]) ) )-1
m = RMSLE(train["Premium Amount"].values, pred)
print(f"Exponented Mean Log 1p produces CV RMSLE = {m}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train.tail()

In [ ]:
scaler = MinMaxScaler()
print(scaler.fit(train))
MinMaxScaler()
print(scaler.data_max_)

In [ ]:
sns.heatmap(scaler.transform(train))

In [ ]:
colunms = ['customer_feedback_Good','occupation_Self-Employed','policy_type_Basic',
           'education_level_PhD','Previous Claims','marital_status_Single','policy_type_Premium',
           "education_level_Bachelor's",'smoking_status_No','Premium Amount', 
           'day_of_week', 'day_sin','day_cos', 'day_of_year','year_day_sin','year_day_cos','month','month_sin','month_cos','seconds_since_1970']

In [ ]:
train = train[:10000]
train

In [ ]:
train = train[colunms]
train

In [ ]:
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.preprocessing import TargetEncoder, FunctionTransformer
from sklearn.model_selection import KFold, cross_val_predict

In [ ]:
#  sdfsdfsdf

In [ ]:
#separando dados do alvo
X = train.drop('Premium Amount', axis=1)
y = train['Premium Amount']

In [ ]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [ ]:
# Divisão do conjunto de dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
X_train = X_train[X_train.columns]
X_train

In [ ]:
X_train.columns.to_list()

In [ ]:
X_train

In [ ]:
#import necessary library
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression

# #by Drawing a plot and recognize it we can find out the suitable dgree.(degree=2)
# poly_reg  = PolynomialFeatures(degree=2)
# X_poly = poly_reg.fit_transform(X_train)

# X_train2, X_test2, y_train2, y_test2 = train_test_split(X_poly, y, test_size=0.20, random_state =1)

# lin_reg = LinearRegression()
# lin_reg  = lin_reg.fit(X_train2,y_train2)

# print(lin_reg.score(X_test2, y_test2))

In [ ]:
n_samples, n_features = 10, 5

lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

lasso_preds = lasso.predict(X_test)
lasso_acc = np.sqrt(mean_squared_error(y_test, lasso_preds))
lasso_acc

In [ ]:
mean_squared_log_error(y_test, lasso_preds)

In [ ]:
n_samples, n_features = 10, 5

clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train)

clf_preds = clf.predict(X_test)
clf_acc = np.sqrt(mean_squared_error(y_test, clf_preds))
clf_acc

In [ ]:
mean_squared_log_error(y_test, clf_preds)

In [ ]:
# Modelo LinearRegression


lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lin_reg_preds = lin_reg.predict(X_test)
lin_reg_acc = np.sqrt(mean_squared_error(y_test, lin_reg_preds))
lin_reg_acc

#lin_reg_acc = accuracy_score(y_test, lin_reg_preds)
#lin_reg_acc

In [ ]:
#mean_squared_log_error(y_test, lin_reg_preds)

In [ ]:
# Modelo LinearRegression scaled

lin_reg_scaled = make_pipeline(StandardScaler(),
                    SGDRegressor(max_iter=10000, tol=1e-3))
lin_reg_scaled.fit(X_train, y_train)
lin_reg_scaled_preds = lin_reg_scaled.predict(X_test)
#lin_reg_scaled_preds


In [ ]:
mean_squared_log_error(y_test, lin_reg_scaled_preds)

In [ ]:
#SGDRegressor
# max_iter=10000, tol=1e-3, loss='squared_error'
reg = SGDRegressor(max_iter=10000, tol=1e-3, alpha=0.1, shuffle=True, loss='squared_error')
reg.fit(X_train, y_train)
reg_preds = reg.predict(X_test)

reg_acc = np.sqrt(mean_squared_error(y_test, reg_preds))
reg_acc

In [ ]:
st.mode(reg_preds).mode

In [ ]:
reg_preds[ reg_preds < 0 ] = 0

In [ ]:
mean_squared_log_error(y_test, reg_preds)

In [ ]:
#SGDRegressor
reg_scaled = make_pipeline(StandardScaler(),
                    SGDRegressor(max_iter=1000, tol=1e-3, shuffle=True, loss='squared_error'))
reg_scaled.fit(X_train, y_train)
reg_scaled_preds = reg_scaled.predict(X_test)

reg_scaled_acc = np.sqrt(mean_squared_error(y_test, reg_scaled_preds))
reg_scaled_acc


In [ ]:
mean_squared_log_error(y_test, reg_scaled_preds)

In [ ]:
fsdfsdf

In [ ]:
# Modelo CatBoost
# cat_model = CatBoostClassifier()
# cat_model.fit(X_train, y_train)
# cat_preds = cat_model.predict(X_test)
# cat_acc = accuracy_score(y_test, cat_preds)
# cat_acc

# lin_reg_acc = np.sqrt(mean_squared_error(y_test, cat_preds))
# lin_reg_acc

In [ ]:
#Modelo CatBoost
cat_model_regressor = CatBoostRegressor()
cat_model_regressor.fit(X_train, y_train)
cat_regressor_preds = cat_model_regressor.predict(X_test)
#cat_acc = accuracy_score(y_test, cat_regressor_preds)
#cat_acc

lin_reg_acc = np.sqrt(mean_squared_error(y_test, cat_regressor_preds))
#lin_reg_acc

In [ ]:
mean_squared_log_error(y_test, cat_regressor_preds)

In [ ]:
# # Modelo LightGBM
# lgb_model = LGBMClassifier()
# lgb_model.fit(X_train, y_train)
# lgb_preds = lgb_model.predict(X_test)
# lgb_acc = accuracy_score(y_test, lgb_preds)
# lgb_acc

In [ ]:
#Modelo LightGBMR

lgb_params = {
    'num_leaves': 71,
    'learning_rate': 0.05412467152424433,
    'n_estimators': 595,
    'max_depth': 12,
    'min_data_in_leaf': 97,
    'bagging_fraction': 0.5200288825838669,
    'feature_fraction': 0.9881738491942492,
    'n_jobs': -1
}

lgbmr_model = LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=0, gpu_device_id=0, verbose=-1)
lgbmr_model.fit(X_train, y_train)
lgbmr_preds = lgbmr_model.predict(X_test)
# lgbmr_acc = accuracy_score(y_test, lgbmr_preds)
mse = mean_squared_error(y_test, lgbmr_preds)
lgbmr_acc = np.sqrt(mse)
lgbmr_acc

In [ ]:
mean_squared_log_error(y_test, lgbmr_preds)

In [ ]:
# Support Vector Machines
"""
svc = SVC()
svc.fit(X_train, y_train)
SVC_pred = svc.predict(X_test)
# acc_svc = round(svc.score(X_train, y_train) * 100, 2)
# acc_svc
mse = mean_squared_error(y_test, SVC_pred)
SVC_acc = np.sqrt(mse)
SVC_acc
"""

In [ ]:
#mean_squared_log_error(y_test, SVC_pred)

In [ ]:
linearSVC = LinearSVC()
linearSVC.fit(X_train, y_train)
linearSVC_pred = linearSVC.predict(X_test)
# acc_svc = round(svc.score(X_train, y_train) * 100, 2)
# acc_svc
mse = mean_squared_error(y_test, linearSVC_pred)
linearSVC_acc = np.sqrt(mse)
linearSVC_acc

In [ ]:
mean_squared_log_error(y_test, linearSVC_pred)

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, y_train)
perceptron_pred = perceptron.predict(X_test)
# acc_perceptron = round(perceptron.score(X_train, y_train) * 100, 2)
# acc_perceptron

mse = mean_squared_error(y_test, perceptron_pred)
perceptron_acc = np.sqrt(mse)
perceptron_acc

In [ ]:
mean_squared_log_error(y_test, perceptron_pred)

In [ ]:
##KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
# acc_knn = round(knn.score(X_train, y_train) * 100, 2)

mse = mean_squared_error(y_test, knn_pred)
acc_knn = np.sqrt(mse)
acc_knn

In [ ]:
mean_squared_log_error(y_test, knn_pred)

In [ ]:
##RandomForestClassifier

#criterion='gini',n_estimators=1100,max_depth=5,min_samples_split=4,min_samples_leaf=5,max_features='auto',oob_score=True,random_state=42,n_jobs=-1,verbose=1



random_forest = RandomForestClassifier(criterion='log_loss', n_estimators=1000, min_samples_split=4)
random_forest.fit(X_train, y_train)
random_forest_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
# acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)

mse = mean_squared_error(y_test, random_forest_pred)
acc_random_forest = np.sqrt(mse)
acc_random_forest

In [ ]:
mean_squared_log_error(y_test, random_forest_pred)

In [ ]:
##RandomForestRegressor

#criterion='gini',n_estimators=1100,max_depth=5,min_samples_split=4,min_samples_leaf=5,max_features='auto',oob_score=True,random_state=42,n_jobs=-1,verbose=1



random_forest_regressor = RandomForestRegressor(criterion='squared_error', n_estimators=1000, max_features='log2', min_samples_split=4)
random_forest_regressor.fit(X_train, y_train)
random_forest_regressor_pred = random_forest_regressor.predict(X_test)
random_forest_regressor.score(X_train, y_train)
acc_random_forest_regressor = round(random_forest_regressor.score(X_train, y_train) * 100, 2)

mse = mean_squared_error(y_test, random_forest_regressor_pred)
acc_random_forest_regressor = np.sqrt(mse)
acc_random_forest_regressor

In [ ]:
mean_squared_log_error(y_test, random_forest_regressor_pred)

In [ ]:
y_train = y_train.astype('int16')

In [ ]:
y_train

In [ ]:
## XGBClassifier

le = LabelEncoder()
y_train = le.fit_transform(y_train)

xgbboost = XGBClassifier(learning_rate=1,max_depth=2)
xgbboost.fit(X_train, y_train)


xgbboost_pred = xgbboost.predict(X_test)
# xgbboost_classifier = round(xgbboost.score(X_train, y_train) * 100, 2)

mse = mean_squared_error(y_test, xgbboost_pred)
xgbboost_classifier = np.sqrt(mse)
xgbboost_classifier

In [ ]:
mean_squared_log_error(y_test, xgbboost_pred)

In [ ]:
## XGBRegressor

le = LabelEncoder()
y_train = le.fit_transform(y_train)

xgb_params = {
    'n_estimators': 1078, 
    'learning_rate': 0.016084079332671603, 
    'max_depth': 10, 
    'min_child_weight': 8, 
    'subsample': 0.8732132237392727, 
    'colsample_bytree': 0.9756972730817159, 
    'reg_alpha': 3.386299962300141, 
    'reg_lambda': 8.964009483088061,
    'enable_categorical': 'True',
    'device': 'cuda'
}

#xgbboostregressor = XGBRegressor(learning_rate=1,max_depth=1, eval_metric='rmse')
xgbboostregressor = XGBRegressor(**xgb_params)
xgbboostregressor.fit(X_train, y_train)


xgbboostregressor_pred = xgbboostregressor.predict(X_test)
# xgbboost_classifier = round(xgbboost.score(X_train, y_train) * 100, 2)

mse = mean_squared_error(y_test, xgbboostregressor_pred)
xgbboost_regressor = np.sqrt(mse)
xgbboost_regressor

In [ ]:
mean_squared_log_error(y_test, xgbboostregressor_pred)

In [ ]:
#GaussianNB

gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
gaussian_pred = gaussian.predict(X_test)
# acc_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)

mse = mean_squared_error(y_test, gaussian_pred)
acc_gaussian = np.sqrt(mse)
acc_gaussian

In [ ]:
mean_squared_log_error(y_test, gaussian_pred)

In [ ]:
X_train.columns

In [ ]:
#GradientBoostingClassifier

# GradientBoostingClassifierObj = GradientBoostingClassifier
# GradientBoostingClassifier = GradientBoostingClassifier()

# GradientBoostingClassifier.fit(X_train, y_train)
# GradientBoostingClassifier_pred = GradientBoostingClassifier.predict(X_test)
# gradient_boosting_classifier = round(GradientBoostingClassifier.score(X_train, y_train) * 100, 2)

# gradient_boosting_classifier


In [ ]:
#linear_regressor.coef_.ravel()

In [ ]:
#print(linear_regressor.coef_)

In [ ]:
print(lin_reg.get_params())

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# def plot_coefficients(classifier, feature_names, top_features=20):
#     # Verifique se classifier.coef_ é 2D (multivariado)
#     coef = classifier.coef_
#     if coef.ndim > 1:
#         raise ValueError("This function only supports single-output regression models. "
#                          "For multi-output, iterate over the targets.")
    
#     # Flatten coefficients
#     coef = coef.ravel()
    
#     # Valide o tamanho de feature_names
#     if len(feature_names) != len(coef):
#         raise ValueError(f"Mismatch: feature_names has {len(feature_names)} elements, "
#                          f"but classifier.coef_ has {len(coef)} elements.")
    
#     # Obter os índices das maiores e menores características
#     top_positive_coefficients = np.argsort(coef)[-top_features:]
#     top_negative_coefficients = np.argsort(coef)[:top_features]
#     top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    
#     # Criar o gráfico
#     plt.figure(figsize=(15, 5))
#     colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
#     plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
#     feature_names = np.array(feature_names)
#     plt.xticks(np.arange(2 * top_features), feature_names[top_coefficients], rotation=60, ha='right')
#     plt.show()
    
    
# cv = CountVectorizer()
# cv.fit(X_train)
# print(len(cv.vocabulary_))
# print( cv.get_feature_names() )
# X_train_v = cv.transform(X_train)

# feature_names = X_train.columns.to_list()


# # random_forest_regressor = RandomForestRegressor(criterion='squared_error', n_estimators=100, min_samples_split=4)
# # random_forest_regressor.fit(X_train, y_train)
# # plot_coefficients(random_forest_regressor, feature_names)

# lin_reg1 = LinearRegression()
# lin_reg1.fit(X_train, y_train)
# plot_coefficients(lin_reg1, feature_names)

In [ ]:
train.tail()

In [ ]:
# Calculate the z-score for each student's height
# smoking_status_No, 
z = np.abs(st.zscore(train['occupation_Self-Employed']))

# Identify outliers as students with a z-score greater than 3
threshold = 3
outliers = train[z > threshold]

# Print the outliers
print(outliers)

In [ ]:
#import folium

# Função para identificar outliers com IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (df[column] < lower_bound) | (df[column] > upper_bound)

# Identificar outliers nas colunas de interesse

df = pd.DataFrame( detect_outliers_iqr(train, 'customer_feedback_Good') )

# # Criar um mapa centralizado nos dados
# map_center = [df['latitude'].mean(), df['longitude'].mean()]
# mymap = folium.Map(location=map_center, zoom_start=12)

# # Adicionar pontos ao mapa
# for _, row in df.iterrows():
#     if row['outlier']:
#         # Outliers em vermelho
#         folium.Marker(
#             location=[row['latitude'], row['longitude']],
#             popup=f"Outlier: {row['value']}",
#             icon=folium.Icon(color="red")
#         ).add_to(mymap)
#     else:
#         # Dados normais em azul
#         folium.Marker(
#             location=[row['latitude'], row['longitude']],
#             popup=f"Valor: {row['value']}",
#             icon=folium.Icon(color="blue")
#         ).add_to(mymap)

# # Salvar o mapa em um arquivo HTML
# mymap.save("outliers_map.html")
# print("Mapa gerado e salvo como 'outliers_map.html'")


In [ ]:
df

In [ ]:
df[df['customer_feedback_Good'] == True]

In [ ]:
df['customer_feedback_Good']

In [ ]:
from sklearn.ensemble import StackingRegressor

# Modelos base
base_models = [
    ('sgd', SGDRegressor(max_iter=10000, tol=1e-3, alpha=0.1, shuffle=True, loss='squared_error')),
    ('RandomForestClassifier', RandomForestRegressor()),
    ('ridge', Ridge(alpha=1.0)),
    ('lasso', Lasso(alpha=0.1)),
    ('CatBoostRegressor', CatBoostRegressor()),
    ('XGBRegressor', XGBRegressor(**xgb_params)),
    ('LGBMRegressor', LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=0, gpu_device_id=0, verbose=-1))
    
]

# Meta-modelo (Blender)
meta_model = LinearRegression()

# Criando o modelo de Stacking
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

# Treinando o modelo de Stacking
stacking_model.fit(X_train, y_train)

# Previsões
stacking_preds = stacking_model.predict(X_test)

# Avaliação do modelo
mse = mean_squared_error(y_test, stacking_preds)
print(f"MSE do modelo de Stacking: {mse:.4f}")

mse = mean_squared_error(y_test, stacking_preds)
stacking = np.sqrt(mse)
#stacking


In [ ]:
mean_squared_log_error(y_test, stacking_preds)

In [ ]:
#print( np.nan_to_num(2.513131531778787313131131381131341837497979494979848494949494) )

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_log_error

def rmsle1(y_true, y_pred):
    # Substituir NaNs e infinitos em y_true e y_pred
    y_true = np.nan_to_num(y_true, nan=0.0, posinf=1e10, neginf=0.0)
    y_pred = np.nan_to_num(y_pred, nan=0.0, posinf=1e10, neginf=0.0)

    # Garantir que os valores previstos sejam não negativos
    y_pred = np.maximum(0, y_pred)

    # Evitar valores muito grandes ou inválidos
    y_true = np.clip(y_true, 0, 1e10)
    y_pred = np.clip(y_pred, 0, 1e10)

    # Calcular RMSLE
    try:
        return np.sqrt(mean_squared_log_error(y_true, y_pred))
    except ValueError as e:
        raise ValueError(f"Erro no cálculo do RMSLE: {e}\n y_true: {y_true}, y_pred: {y_pred}")


In [ ]:
from sklearn.ensemble import VotingRegressor


lgb_params = {
    'num_leaves': 71,
    'learning_rate': 0.05412467152424433,
    'n_estimators': 595,
    'max_depth': 12,
    'min_data_in_leaf': 97,
    'bagging_fraction': 0.5200288825838669,
    'feature_fraction': 0.9881738491942492,
    'n_jobs': -1
}

xgb_params = {
    'n_estimators': 1078, 
    'learning_rate': 0.016084079332671603, 
    'max_depth': 10, 
    'min_child_weight': 8, 
    'subsample': 0.8732132237392727, 
    'colsample_bytree': 0.9756972730817159, 
    'reg_alpha': 3.386299962300141, 
    'reg_lambda': 8.964009483088061,
    'enable_categorical': 'True',
    'device': 'cuda'
}

def train_model():
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof = np.zeros(len(X))
    models = []

    for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}")
        X_train_voting, X_valid_voting = X.iloc[train_idx], X.iloc[valid_idx]
        y_train_voting, y_valid_voting = y.iloc[train_idx], y.iloc[valid_idx]

        lgbm_model = LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=0, gpu_device_id=0, verbose=-1)

        xgb_model = XGBRegressor(**xgb_params)

        voting_model = VotingRegressor(
            estimators= [
                ('sgd', SGDRegressor(max_iter=10000, tol=1e-3, alpha=0.1, shuffle=True, loss='squared_error')),
                ('RandomForestClassifier', RandomForestRegressor()),
                ('ridge', Ridge(alpha=1.0)),
                ('lasso', Lasso(alpha=0.1)),
                ('CatBoostRegressor', CatBoostRegressor()),
                ('XGBRegressor', XGBRegressor(**xgb_params)),
                ('LGBMRegressor', LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=0, gpu_device_id=0, verbose=-1))
                
            ]
        )

        voting_model.fit(X_train_voting, y_train_voting)
        oof[valid_idx] = np.nan_to_num( np.maximum(0, voting_model.predict(X_valid_voting)) )
        fold_rmsle = np.nan_to_num( rmsle1(np.expm1(y_valid_voting), np.expm1(oof[valid_idx])) )
        print(f"Fold {fold + 1} RMSLE: {fold_rmsle}")
        models.append(voting_model)

        
    return models, oof


In [ ]:
models,oof = train_model()

In [ ]:
models

In [ ]:
#oof = oof.astype('float64')
#train['Premium Amount'] = train['Premium Amount'].astype('float64')

In [ ]:
#train['Premium Amount'][train['Premium Amount'] <= 0]

In [ ]:
#oof = np.float64(oof)
#train['Premium Amount'] = np.float64()

In [ ]:
rmsle1 = rmsle1(np.expm1(train['Premium Amount']), np.expm1(oof))
print( rmsle1 )

In [ ]:
mean_squared_log_error(y_test, stacking_preds)

In [ ]:
mean_squared_log_error(y_test, lasso_preds)

In [ ]:
mean_squared_log_error(y_test, clf_preds)

In [ ]:
mean_squared_log_error(y_test, lin_reg_preds)

In [ ]:
mean_squared_log_error(y_test, lin_reg_scaled_preds)

In [ ]:
mean_squared_log_error(y_test, reg_preds)

In [ ]:
mean_squared_log_error(y_test, reg_scaled_preds)

In [ ]:
# mean_squared_log_error(y_test, cat_preds)

In [ ]:
#mean_squared_log_error(y_test, lgb_preds)

In [ ]:
mean_squared_log_error(y_test, lgbmr_preds)

In [ ]:
mean_squared_log_error(y_test, linearSVC_pred)

In [ ]:
mean_squared_log_error(y_test, SVC_pred)

In [ ]:
mean_squared_log_error(y_test, perceptron_pred)

In [ ]:
mean_squared_log_error(y_test, knn_pred)

In [ ]:
mean_squared_log_error(y_test, random_forest_pred)

In [ ]:
mean_squared_log_error(y_test, random_forest_regressor_pred)

In [ ]:
mean_squared_log_error(y_test, xgbboost_pred)

In [ ]:
mean_squared_log_error(y_test, xgbboostregressor_pred)

In [ ]:
mean_squared_log_error(y_test, gaussian_pred)

In [ ]:
#mean_squared_log_error(y_test, GradientBoostingClassifier_pred)

In [ ]:
test[X.columns]

In [ ]:
submission1 = reg_scaled.predict(train[X.columns])
#submission1

In [ ]:
#Last submission in the best model found.

submission = reg_scaled.predict(test[X.columns])
submission

In [ ]:
submission1.shape

In [ ]:
#Submission must be done within the standard requested by the challenge. 
#    In this case, the depression column must be submitted with 0 and 1.
#    This is why it is necessary to convert.

#submission = submission.astype('int64')
submission

In [ ]:
test['id']

In [ ]:
id = test['id'].T.drop_duplicates().T


In [ ]:
id

In [ ]:
reg_scaled_preds.shape

In [ ]:
submission = reg_scaled_preds

In [ ]:
reg_scaled_preds.shape

In [ ]:
dd = pd.DataFrame({

    "id":id,

    "Premium Amount":submission1

})

In [ ]:
dd

In [ ]:
## Submit notebooks to the challenge. Final


submission_final = pd.DataFrame({

        "id":id,

        "Premium Amount":submission

    })

submission_final.to_csv('Arquivo_submissionv2.csv', index=False)


print(" Arquivo submission v2.csv pronto ")

In [ ]:
print('end...')